In [5]:
import json
import torch
# from datasets import load_dataset, load_metric, Dataset
from transformers import BertTokenizerFast, BertForTokenClassification, Trainer, TrainingArguments
from transformers import DataCollatorForTokenClassification



In [1]:
!pip install datasets evaluate

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.8 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset, Dataset

In [3]:
import evaluate

In [ ]:
# Load the dataset
def load_ner_dataset(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

dataset_file = '/content/balanced_conditions_ner_records.json'
dataset = load_ner_dataset(dataset_file)

In [6]:
# Load the dataset
def load_ner_dataset(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

dataset_file = '/content/generated_ner_dataset_5000.json'
dataset = load_ner_dataset(dataset_file)

In [7]:
# Load the tokenizer and model from Hugging Face
model_checkpoint = "bert-base-cased"
tokenizer = BertTokenizerFast.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
label_list = ["O", "B-PERSON", "I-PERSON", "B-PERSON_ARABIC", "I-PERSON_ARABIC", "B-ALIAS", "I-ALIAS",
              "B-DATE_OF_BIRTH", "I-DATE_OF_BIRTH", "B-PLACE_OF_BIRTH", "I-PLACE_OF_BIRTH",
              "B-COUNTRY", "I-COUNTRY", "B-FUNCTION", "I-FUNCTION", "B-RANK", "I-RANK",
              "B-NATIONALITY", "I-NATIONALITY", "B-GENDER", "I-GENDER", "B-PASSPORT_NUMBER",
              "I-PASSPORT_NUMBER", "B-REASON_FOR_LISTING", "I-REASON_FOR_LISTING", "B-DATE_OF_LISTING",
              "I-DATE_OF_LISTING", "B-UN_LISTING_DATE", "I-UN_LISTING_DATE"]

label_to_id = {label: i for i, label in enumerate(label_list)}

# Convert dataset into Hugging Face Dataset format
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128, return_offsets_mapping=True)

    labels = []
    for i, entities in enumerate(examples['entities']):
        offset_mapping = tokenized_inputs['offset_mapping'][i]
        label_ids = [0] * len(offset_mapping)  # Default to "O" (outside entity)

        for entity in entities:
            entity_start = entity['start']
            entity_end = entity['end']
            entity_label = entity['label']

            for idx, (start, end) in enumerate(offset_mapping):
                # If token is within the entity span, assign the appropriate label
                if start >= entity_start and end <= entity_end:
                    if start == entity_start:  # Beginning of the entity
                        label_ids[idx] = label_to_id[f"B-{entity_label}"]
                    else:  # Inside the entity
                        label_ids[idx] = label_to_id[f"I-{entity_label}"]

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    tokenized_inputs.pop("offset_mapping")  # Remove offset mappings after use
    return tokenized_inputs

# Convert dataset to Hugging Face Dataset object
def convert_to_hf_dataset(data):
    texts = [item['text'] for item in data]
    entities = [item['entities'] for item in data]
    return Dataset.from_dict({'text': texts, 'entities': entities})

hf_dataset = convert_to_hf_dataset(dataset)
tokenized_dataset = hf_dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

KeyError: 'B-NAME'

In [8]:
# Update the label list with the new set of labels
label_list = [
    "O",
    "B-NAME", "I-NAME",
    "B-NAME_ARABIC", "I-NAME_ARABIC",
    "B-ALSO_KNOWN_AS", "I-ALSO_KNOWN_AS",
    "B-DATE_OF_BIRTH", "I-DATE_OF_BIRTH",
    "B-PLACE_OF_BIRTH", "I-PLACE_OF_BIRTH",
    "B-NATIONALITY", "I-NATIONALITY",
    "B-PASSPORT_NUMBER", "I-PASSPORT_NUMBER",
    "B-ADDRESS", "I-ADDRESS",
    "B-POSITION", "I-POSITION",
    "B-RANK", "I-RANK",
    "B-REASONS", "I-REASONS",
    "B-GENDER", "I-GENDER",
    "B-DATE_OF_LISTING", "I-DATE_OF_LISTING",
    "B-NATIONAL_ID", "I-NATIONAL_ID"
]

# Create a mapping from labels to IDs
label_to_id = {label: i for i, label in enumerate(label_list)}

# Function to tokenize and align labels for the Hugging Face Dataset
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples['text'],
        truncation=True,
        padding='max_length',
        max_length=128,
        return_offsets_mapping=True
    )

    labels = []
    for i, entities in enumerate(examples['entities']):
        offset_mapping = tokenized_inputs['offset_mapping'][i]
        label_ids = [0] * len(offset_mapping)  # Default to "O" (outside entity)

        for entity in entities:
            entity_start = entity['start']
            entity_end = entity['end']
            entity_label = entity['label']

            for idx, (start, end) in enumerate(offset_mapping):
                # If token is within the entity span, assign the appropriate label
                if start >= entity_start and end <= entity_end:
                    if start == entity_start:  # Beginning of the entity
                        label_ids[idx] = label_to_id[f"B-{entity_label}"]
                    else:  # Inside the entity
                        label_ids[idx] = label_to_id[f"I-{entity_label}"]

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    tokenized_inputs.pop("offset_mapping")  # Remove offset mappings after use
    return tokenized_inputs

# Convert dataset to Hugging Face Dataset object
def convert_to_hf_dataset(data):
    texts = [item['text'] for item in data]
    entities = [item['entities'] for item in data]
    return Dataset.from_dict({'text': texts, 'entities': entities})

# Generate Hugging Face Dataset object and tokenize it
hf_dataset = convert_to_hf_dataset(dataset)
tokenized_dataset = hf_dataset.map(tokenize_and_align_labels, batched=True)


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
# Define the new label list with simplified labels for name, also known as, and date of birth
label_list = ["O", "B-NAME", "I-NAME", "B-ALSO_KNOWN_AS", "I-ALSO_KNOWN_AS",
              "B-DATE_OF_BIRTH", "I-DATE_OF_BIRTH"]

label_to_id = {label: i for i, label in enumerate(label_list)}

# Convert dataset into Hugging Face Dataset format and align labels
def tokenize_and_align_labels(examples):
    # Tokenize inputs
    tokenized_inputs = tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128, return_offsets_mapping=True)

    labels = []
    for i, entities in enumerate(examples['entities']):
        offset_mapping = tokenized_inputs['offset_mapping'][i]
        label_ids = [0] * len(offset_mapping)  # Default to "O" (outside entity)

        for entity in entities:
            entity_start = entity['start']
            entity_end = entity['end']
            entity_label = entity['label']

            for idx, (start, end) in enumerate(offset_mapping):
                # If token is within the entity span, assign the appropriate label
                if start >= entity_start and end <= entity_end:
                    if start == entity_start:  # Beginning of the entity
                        label_ids[idx] = label_to_id[f"B-{entity_label}"]
                    else:  # Inside the entity
                        label_ids[idx] = label_to_id[f"I-{entity_label}"]

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    tokenized_inputs.pop("offset_mapping")  # Remove offset mappings after use
    return tokenized_inputs

# Convert dataset to Hugging Face Dataset object
def convert_to_hf_dataset(data):
    texts = [item['text'] for item in data]
    entities = [item['entities'] for item in data]
    return Dataset.from_dict({'text': texts, 'entities': entities})

# Assuming 'dataset' contains the data in dictionary format
hf_dataset = convert_to_hf_dataset(dataset)
tokenized_dataset = hf_dataset.map(tokenize_and_align_labels, batched=True)


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

KeyError: 'B-OTHER_INFORMATION'

In [ ]:
# Define the new label list with simplified labels for name, also known as, date of birth, and other information
label_list = ["O", "B-NAME", "I-NAME", "B-ALSO_KNOWN_AS", "I-ALSO_KNOWN_AS",
              "B-DATE_OF_BIRTH", "I-DATE_OF_BIRTH", "B-OTHER_INFORMATION", "I-OTHER_INFORMATION"]

label_to_id = {label: i for i, label in enumerate(label_list)}

# Convert dataset into Hugging Face Dataset format and align labels
def tokenize_and_align_labels(examples):
    # Tokenize inputs
    tokenized_inputs = tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128, return_offsets_mapping=True)

    labels = []
    for i, entities in enumerate(examples['entities']):
        offset_mapping = tokenized_inputs['offset_mapping'][i]
        label_ids = [0] * len(offset_mapping)  # Default to "O" (outside entity)

        for entity in entities:
            entity_start = entity['start']
            entity_end = entity['end']
            entity_label = entity['label']

            for idx, (start, end) in enumerate(offset_mapping):
                # If token is within the entity span, assign the appropriate label
                if start >= entity_start and end <= entity_end:
                    if start == entity_start:  # Beginning of the entity
                        label_ids[idx] = label_to_id[f"B-{entity_label}"]
                    else:  # Inside the entity
                        label_ids[idx] = label_to_id[f"I-{entity_label}"]

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    tokenized_inputs.pop("offset_mapping")  # Remove offset mappings after use
    return tokenized_inputs

# Convert dataset to Hugging Face Dataset object
def convert_to_hf_dataset(data):
    texts = [item['text'] for item in data]
    entities = [item['entities'] for item in data]
    return Dataset.from_dict({'text': texts, 'entities': entities})

# Assuming 'balanced_records' contains the data in dictionary format
hf_dataset = convert_to_hf_dataset(dataset)
tokenized_dataset = hf_dataset.map(tokenize_and_align_labels, batched=True)


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [12]:
model = BertForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))




model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Set up TrainingArguments for fine-tuning
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=8,
    weight_decay=0.01,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Data collator to handle padding of sequences
data_collator = DataCollatorForTokenClassification(tokenizer)



In [9]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=1ddb088e312eba7f064df8c995a02ea7ffe2351259e6c60c9f8f65972d798584
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [10]:
metric = evaluate.load("seqeval")

In [ ]:
def compute_metrics(p):
    predictions, labels = p
    predictions = torch.argmax(predictions, dim=2)

    true_labels = [[label_list[l] for l in label if l != -100] for label in labels]
    true_predictions = [[label_list[pred] for (pred, label) in zip(pred, label) if label != -100]
                        for pred, label in zip(predictions, labels)]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
# Define the metric using 'evaluate' library
import evaluate
metric = evaluate.load("seqeval")

label_list = ["O", "B-PERSON", "I-PERSON", "B-PERSON_ARABIC", "I-PERSON_ARABIC", "B-ALIAS", "I-ALIAS",
              "B-DATE_OF_BIRTH", "I-DATE_OF_BIRTH", "B-PLACE_OF_BIRTH", "I-PLACE_OF_BIRTH",
              "B-COUNTRY", "I-COUNTRY", "B-FUNCTION", "I-FUNCTION", "B-RANK", "I-RANK",
              "B-NATIONALITY", "I-NATIONALITY", "B-GENDER", "I-GENDER", "B-PASSPORT_NUMBER",
              "I-PASSPORT_NUMBER", "B-REASON_FOR_LISTING", "I-REASON_FOR_LISTING", "B-DATE_OF_LISTING",
              "I-DATE_OF_LISTING", "B-UN_LISTING_DATE", "I-UN_LISTING_DATE"]

# Load the metric
metric = evaluate.load("seqeval")

# Define the compute_metrics function
def compute_metrics(p):
    # Ensure predictions are tensors
    predictions = torch.tensor(p.predictions)

    # Convert label_ids (true labels) to tensors
    labels = torch.tensor(p.label_ids)

    # Apply argmax to get the predicted class IDs
    predictions = torch.argmax(predictions, dim=2)

    # Convert predictions and true labels to list of label strings
    true_labels = [
        [label_list[l.item()] for l in label if l != -100]  # Convert to string labels, excluding -100 (ignored tokens)
        for label in labels
    ]

    true_predictions = [
        [label_list[p.item()] for (p, l) in zip(pred, label) if l != -100]  # Convert to string labels, excluding -100
        for pred, label in zip(predictions, labels)
    ]

    # Compute metrics using seqeval metric
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
)

# Create a Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,  # Assuming tokenized_dataset is your training dataset
    eval_dataset=tokenized_dataset,   # Assuming tokenized_dataset is also your eval dataset
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Start training
trainer.train()

# Save the final model and tokenizer
trainer.save_model("./ner_model")
tokenizer.save_pretrained("./ner_model")

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.037412,0.981656,0.987095,0.984368,0.995148
2,No log,0.010930,0.993100,0.993100,0.993100,0.997898
3,No log,0.008746,0.993929,0.993929,0.993929,0.998070
4,No log,0.007091,0.994327,0.994327,0.994327,0.998172
5,No log,0.005236,0.996849,0.996915,0.996882,0.998805
6,No log,0.004032,0.998441,0.998474,0.998457,0.999188
7,No log,0.003627,0.998773,0.998806,0.998789,0.999258
8,0.113900,0.003198,0.999237,0.999270,0.999254,0.999414
9,0.113900,0.002776,0.999469,0.999469,0.999469,0.999484
10,0.113900,0.002674,0.999469,0.999469,0.999469,0.999508


('./ner_model/tokenizer_config.json',
 './ner_model/special_tokens_map.json',
 './ner_model/vocab.txt',
 './ner_model/added_tokens.json',
 './ner_model/tokenizer.json')

In [ ]:
import random
import json
from faker import Faker

fake = Faker()

def generate_record(variant="name_only"):
    """
    Generate a single record based on the variant provided.
    """
    name = fake.name()
    aka = f"(also known as {fake.name()})"
    dob = fake.date_of_birth().strftime("%d %b %Y")
    company_name = fake.company()

    if variant == "name_only":
        text = f"Name: {name}"
        entities = [
            {"start": text.find(name), "end": text.find(name) + len(name), "label": "NAME"}
        ]
    elif variant == "name_and_aka":
        text = f"Name: {name} {aka}"
        entities = [
            {"start": text.find(name), "end": text.find(name) + len(name), "label": "NAME"},
            {"start": text.find(aka), "end": text.find(aka) + len(aka), "label": "ALSO_KNOWN_AS"}
        ]
    elif variant == "name_aka_dob":
        text = f"Name: {name} {aka}, DOB: {dob}"
        entities = [
            {"start": text.find(name), "end": text.find(name) + len(name), "label": "NAME"},
            {"start": text.find(aka), "end": text.find(aka) + len(aka), "label": "ALSO_KNOWN_AS"},
            {"start": text.find(dob), "end": text.find(dob) + len(dob), "label": "DATE_OF_BIRTH"}
        ]
    elif variant == "company_only":
        text = f"{company_name}"
        entities = [
            {"start": text.find(company_name), "end": text.find(company_name) + len(company_name), "label": "NAME"}
        ]
    elif variant == "company_and_aka":
        text = f"{company_name} (also known as {fake.company()})"
        aka_part = text.split("(also known as ")[-1].rstrip(")")
        entities = [
            {"start": text.find(company_name), "end": text.find(company_name) + len(company_name), "label": "NAME"},
            {"start": text.find(aka_part), "end": text.find(aka_part) + len(aka_part), "label": "ALSO_KNOWN_AS"}
        ]

    return {"text": text, "entities": entities}

def create_balanced_dataset(n=10000):
    """
    Create a balanced dataset with 10,000 records.
    """
    dataset = []

    # Define the number of records for each variant type
    records_per_variant = n // 5  # 5 types of records

    for _ in range(records_per_variant):
        dataset.append(generate_record("name_only"))
        dataset.append(generate_record("name_and_aka"))
        dataset.append(generate_record("name_aka_dob"))
        dataset.append(generate_record("company_only"))
        dataset.append(generate_record("company_and_aka"))

    return dataset

# Generate the dataset
dataset = create_balanced_dataset(n=10000)

# Save to JSON file
file_path = "balanced_ner_dataset_10k.json"
with open(file_path, 'w') as f:
    json.dump(dataset, f)

print(f"Dataset saved to {file_path}")


Dataset saved to balanced_ner_dataset_10k.json


In [ ]:
# Define the metric using 'evaluate' library
import evaluate
import torch

# Simplified label list based on the requested fields: Name, Also Known As, Date of Birth
label_list = ["O", "B-NAME", "I-NAME", "B-ALSO_KNOWN_AS", "I-ALSO_KNOWN_AS",
              "B-DATE_OF_BIRTH", "I-DATE_OF_BIRTH","B-OTHER_INFORMATION", "I-OTHER_INFORMATION"]

# Load the metric for sequence evaluation
metric = evaluate.load("seqeval")

# Define the compute_metrics function
def compute_metrics(p):
    # Ensure predictions are tensors
    predictions = torch.tensor(p.predictions)

    # Convert label_ids (true labels) to tensors
    labels = torch.tensor(p.label_ids)

    # Apply argmax to get the predicted class IDs
    predictions = torch.argmax(predictions, dim=2)

    # Convert predictions and true labels to list of label strings
    true_labels = [
        [label_list[l.item()] for l in label if l != -100]  # Convert to string labels, excluding -100 (ignored tokens)
        for label in labels
    ]

    true_predictions = [
        [label_list[p.item()] for (p, l) in zip(pred, label) if l != -100]  # Convert to string labels, excluding -100
        for pred, label in zip(predictions, labels)
    ]

    # Compute metrics using seqeval metric
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

# Create a Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,  # Assuming tokenized_dataset is your training dataset
    eval_dataset=tokenized_dataset,   # Assuming tokenized_dataset is also your eval dataset
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Start training
trainer.train()

# Save the final model and tokenizer
trainer.save_model("./ner_model_new_trail")
tokenizer.save_pretrained("./ner_model_new_trail")


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.000170,1.000000,1.000000,1.000000,1.000000
2,0.036900,0.000079,1.000000,1.000000,1.000000,1.000000
3,0.036900,0.000057,1.000000,1.000000,1.000000,1.000000
4,0.000200,0.000047,1.000000,1.000000,1.000000,1.000000
5,0.000100,0.000045,1.000000,1.000000,1.000000,1.000000


('./ner_model_new_trail/tokenizer_config.json',
 './ner_model_new_trail/special_tokens_map.json',
 './ner_model_new_trail/vocab.txt',
 './ner_model_new_trail/added_tokens.json',
 './ner_model_new_trail/tokenizer.json')

In [ ]:
# Define the updated label list with the new entity types
label_list = [
    "O",
    "B-NAME", "I-NAME",
    "B-NAME_ARABIC", "I-NAME_ARABIC",
    "B-ALSO_KNOWN_AS", "I-ALSO_KNOWN_AS",
    "B-DATE_OF_BIRTH", "I-DATE_OF_BIRTH",
    "B-PLACE_OF_BIRTH", "I-PLACE_OF_BIRTH",
    "B-NATIONALITY", "I-NATIONALITY",
    "B-PASSPORT_NUMBER", "I-PASSPORT_NUMBER",
    "B-ADDRESS", "I-ADDRESS",
    "B-POSITION", "I-POSITION",
    "B-RANK", "I-RANK",
    "B-REASONS", "I-REASONS",
    "B-GENDER", "I-GENDER",
    "B-DATE_OF_LISTING", "I-DATE_OF_LISTING",
    "B-NATIONAL_ID", "I-NATIONAL_ID"
]

# Load the metric for evaluation
import evaluate
metric = evaluate.load("seqeval")

# Define the function to compute evaluation metrics
def compute_metrics(p):
    # Ensure predictions are tensors
    predictions = torch.tensor(p.predictions)

    # Convert label_ids (true labels) to tensors
    labels = torch.tensor(p.label_ids)

    # Apply argmax to get the predicted class IDs
    predictions = torch.argmax(predictions, dim=2)

    # Convert predictions and true labels to list of label strings
    true_labels = [
        [label_list[l.item()] for l in label if l != -100]  # Convert to string labels, excluding -100 (ignored tokens)
        for label in labels
    ]

    true_predictions = [
        [label_list[p.item()] for (p, l) in zip(pred, label) if l != -100]  # Convert to string labels, excluding -100
        for pred, label in zip(predictions, labels)
    ]

    # Compute metrics using seqeval metric
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

# Set up training arguments
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

# Create a Trainer object
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,  # Assuming tokenized_dataset is your training dataset
    eval_dataset=tokenized_dataset,   # Assuming tokenized_dataset is also your eval dataset
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Start training
trainer.train()

# Save the final model and tokenizer
trainer.save_model("./ner_model_EU")
tokenizer.save_pretrained("./ner_model_EU")


Epoch,Training Loss,Validation Loss


In [ ]:
from transformers import BertTokenizerFast, BertForTokenClassification
import torch

# Load the fine-tuned model and tokenizer
model_dir = "./ner_model_new"  # Directory where the fine-tuned model is saved
tokenizer = BertTokenizerFast.from_pretrained(model_dir)
model = BertForTokenClassification.from_pretrained(model_dir)

# Set the model to evaluation mode
model.eval()

# Define the string to perform NER on
x = """
Htoo Group of Companies (also known among other names as Htoo Trading Company Limited and Htoo Trading Group Company)"""
# Tokenize the text in variable x
inputs = tokenizer(x, return_tensors="pt", truncation=True, padding=True, is_split_into_words=False)

# Perform NER using the model
with torch.no_grad():
    outputs = model(**inputs)

# Get predictions (logits) and convert them into label IDs
predictions = torch.argmax(outputs.logits, dim=2)

# List of entity labels (ensure it's the same as used during fine-tuning)
label_list = ["O", "B-PERSON", "I-PERSON", "B-PERSON_ARABIC", "I-PERSON_ARABIC", "B-ALIAS", "I-ALIAS",
              "B-DATE_OF_BIRTH", "I-DATE_OF_BIRTH", "B-PLACE_OF_BIRTH", "I-PLACE_OF_BIRTH",
              "B-COUNTRY", "I-COUNTRY", "B-FUNCTION", "I-FUNCTION", "B-RANK", "I-RANK",
              "B-NATIONALITY", "I-NATIONALITY", "B-GENDER", "I-GENDER", "B-PASSPORT_NUMBER",
              "I-PASSPORT_NUMBER", "B-REASON_FOR_LISTING", "I-REASON_FOR_LISTING", "B-DATE_OF_LISTING",
              "I-DATE_OF_LISTING", "B-UN_LISTING_DATE", "I-UN_LISTING_DATE"]

# Convert token IDs back to tokens
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

# Convert predicted label IDs back to label names
predicted_labels = [label_list[p.item()] for p in predictions[0]]

# Format the output as you requested
output = []
for token, label in zip(tokens, predicted_labels):
    if label != "O":  # Skip 'O' (non-entity tokens)
        output.append(f"{token} ['{label}']")
    else:
        output.append(token)

# Join the tokens into a sentence with spaces
output_text = " ".join(output)
print(output_text)


[CLS] ['B-PERSON'] H ['B-REASON_FOR_LISTING'] ##to ['I-PERSON'] ##o ['I-PERSON'] Group ['I-REASON_FOR_LISTING'] of ['I-REASON_FOR_LISTING'] Companies ['I-REASON_FOR_LISTING'] ( also known among other names as H ['B-REASON_FOR_LISTING'] ##to ['B-REASON_FOR_LISTING'] ##o ['I-FUNCTION'] Trading ['I-FUNCTION'] Company ['I-FUNCTION'] Limited ['I-FUNCTION'] and H ['B-REASON_FOR_LISTING'] ##to ['I-FUNCTION'] ##o ['I-FUNCTION'] Trading ['I-FUNCTION'] Group ['I-FUNCTION'] Company ['I-FUNCTION'] ) [SEP] ['B-PERSON']


In [ ]:
from transformers import BertTokenizerFast, BertForTokenClassification
import torch

# Load the fine-tuned model and tokenizer
model_dir = "./ner_model_new"  # Directory where the fine-tuned model is saved
tokenizer = BertTokenizerFast.from_pretrained(model_dir)
model = BertForTokenClassification.from_pretrained(model_dir)

# Set the model to evaluation mode
model.eval()

# Define the string to perform NER on
x = """
Suntac Group of Companies (also known as Suntac Technologies)"""

# Tokenize the text in variable x
inputs = tokenizer(x, return_tensors="pt", truncation=True, padding=True, is_split_into_words=False)

# Perform NER using the model
with torch.no_grad():
    outputs = model(**inputs)

# Get predictions (logits) and convert them into label IDs
predictions = torch.argmax(outputs.logits, dim=2)

# Simplified label list without B- and I- prefixes
label_list = ["O", "NAME", "NAME", "ALSO_KNOWN_AS", "ALSO_KNOWN_AS", "DATE_OF_BIRTH", "DATE_OF_BIRTH"]

# Convert token IDs back to tokens
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

# Convert predicted label IDs back to label names (without B- and I- prefixes)
predicted_labels = [label_list[p.item()] for p in predictions[0]]

# Group consecutive tokens with the same label
output = []
current_entity = []
current_label = None

for token, label in zip(tokens, predicted_labels):
    if label != "O":  # If it's an entity label
        if label == current_label:  # If the label is the same as the previous one, continue collecting tokens
            current_entity.append(token)
        else:
            if current_entity:  # If we have a previous entity, finalize it
                output.append(f"{' '.join(current_entity)} ['{current_label}']")
            current_entity = [token]  # Start a new entity
            current_label = label
    else:
        if current_entity:  # If we encounter "O" and have an ongoing entity, finalize it
            output.append(f"{' '.join(current_entity)} ['{current_label}']")
            current_entity = []
            current_label = None

# Handle the last entity if it exists
if current_entity:
    output.append(f"{' '.join(current_entity)} ['{current_label}']")

# Join the tokens into a sentence with spaces
output_text = " ".join(output)
print(output_text)


Sun ['ALSO_KNOWN_AS'] ##ta ##c ['NAME'] Group ['ALSO_KNOWN_AS'] Companies ['ALSO_KNOWN_AS'] Sun ['ALSO_KNOWN_AS'] ##ta ['NAME'] ##c Technologies ['ALSO_KNOWN_AS']


In [ ]:
from transformers import BertTokenizerFast, BertForTokenClassification
import torch

# Load the fine-tuned model and tokenizer
model_dir = "./ner_model_new"  # Directory where the fine-tuned model is saved
tokenizer = BertTokenizerFast.from_pretrained(model_dir)
model = BertForTokenClassification.from_pretrained(model_dir)

# Set the model to evaluation mode
model.eval()

# Define the string to perform NER on
x = """
Chan Nyein, born 1944 (also known among other names as Chang Nyein)"""

# Tokenize the text in variable x
inputs = tokenizer(x, return_tensors="pt", truncation=True, padding=True, is_split_into_words=False)

# Perform NER using the model
with torch.no_grad():
    outputs = model(**inputs)

# Get predictions (logits) and convert them into label IDs
predictions = torch.argmax(outputs.logits, dim=2)

# Updated label list to match your simplified entity categories
label_list = ["O", "B-NAME", "I-NAME", "B-ALSO_KNOWN_AS", "I-ALSO_KNOWN_AS", "B-DATE_OF_BIRTH", "I-DATE_OF_BIRTH"]

# Convert token IDs back to tokens
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

# Convert predicted label IDs back to label names
predicted_labels = [label_list[p.item()] for p in predictions[0]]

# Format the output as requested
output = []
for token, label in zip(tokens, predicted_labels):
    if label != "O":  # Skip 'O' (non-entity tokens)
        output.append(f"{token} ['{label}']")
    else:
        output.append(token)

# Join the tokens into a sentence with spaces
output_text = " ".join(output)
print(output_text)


[CLS] Chan ['B-NAME'] N ['I-NAME'] ##ye ['I-NAME'] ##in ['I-NAME'] , born 1944 ['I-DATE_OF_BIRTH'] ( also known among other names as Chang ['I-ALSO_KNOWN_AS'] N ['I-ALSO_KNOWN_AS'] ##ye ['I-ALSO_KNOWN_AS'] ##in ['I-ALSO_KNOWN_AS'] ) ['I-ALSO_KNOWN_AS'] [SEP]


In [ ]:
from transformers import BertTokenizerFast, BertForTokenClassification
import torch

# Load the fine-tuned model and tokenizer
model_dir = "./ner_model_new_trail"  # Directory where the fine-tuned model is saved
tokenizer = BertTokenizerFast.from_pretrained(model_dir)
model = BertForTokenClassification.from_pretrained(model_dir)

# Set the model to evaluation mode
model.eval()

# Define the string to perform NER on
x = """Joshua Nishanth"""

# Tokenize the text in variable x
inputs = tokenizer(x, return_tensors="pt", truncation=True, padding=True, is_split_into_words=False)

# Perform NER using the model
with torch.no_grad():
    outputs = model(**inputs)

# Get predictions (logits) and convert them into label IDs
predictions = torch.argmax(outputs.logits, dim=2)

# Simplified label list without B- and I- prefixes
label_list = ["O", "NAME", "NAME", "ALSO_KNOWN_AS", "ALSO_KNOWN_AS", "DATE_OF_BIRTH", "DATE_OF_BIRTH","OTHER_INFORMAION", "OTHER_INFORMATION"]

# Convert token IDs back to tokens
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

# Convert predicted label IDs back to label names (without B- and I- prefixes)
predicted_labels = [label_list[p.item()] for p in predictions[0]]

# Group subword tokens and their labels
output = []
current_entity = []
current_label = None

for token, label in zip(tokens, predicted_labels):
    if token.startswith("##"):  # Handle subword tokens
        current_entity[-1] = current_entity[-1] + token[2:]  # Merge subword with previous token
    elif label != "O":  # If it's an entity label
        if label == current_label:  # Continue collecting tokens for the same entity
            current_entity.append(token)
        else:
            if current_entity:  # If there's a previous entity, finalize it
                output.append(f"{' '.join(current_entity)} ['{current_label}']")
            current_entity = [token]  # Start a new entity
            current_label = label
    else:
        if current_entity:  # If we encounter "O" and have an ongoing entity, finalize it
            output.append(f"{' '.join(current_entity)} ['{current_label}']")
            current_entity = []
            current_label = None

# Handle the last entity if it exists
if current_entity:
    output.append(f"{' '.join(current_entity)} ['{current_label}']")

# Join the tokens into a sentence with spaces
output_text = " ".join(output)
print(output_text)


[CLS] Joshua Nishanth [SEP] ['NAME']


In [ ]:
import shutil
import os

# Source folder in Colab
source_folder = '/content/ner_model_EU"'

# Destination in Google Drive
destination_folder = '/content/drive/MyDrive/NER'

# Create the destination folder if it doesn't exist
os.makedirs(destination_folder, exist_ok=True)

# Copy the folder
shutil.copytree(source_folder, destination_folder, dirs_exist_ok=True)

FileNotFoundError: [Errno 2] No such file or directory: '/content/ner_model_EU"'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from transformers import BertTokenizerFast, BertForTokenClassification
import torch

# Load the fine-tuned model and tokenizer
model_dir = "./ner_model_new"  # Directory where the fine-tuned model is saved
tokenizer = BertTokenizerFast.from_pretrained(model_dir)
model = BertForTokenClassification.from_pretrained(model_dir)

# Set the model to evaluation mode
model.eval()

# Define the string to perform NER on
x = """Khin Maung Swe"""

# Tokenize the text in variable x
inputs = tokenizer(x, return_tensors="pt", truncation=True, padding=True, is_split_into_words=False)

# Perform NER using the model
with torch.no_grad():
    outputs = model(**inputs)

# Get predictions (logits) and convert them into label IDs
predictions = torch.argmax(outputs.logits, dim=2)

# Full label list including B- and I- prefixes for the different entities
label_list = ["O", "B-NAME", "I-NAME", "B-ALSO_KNOWN_AS", "I-ALSO_KNOWN_AS",
              "B-DATE_OF_BIRTH", "I-DATE_OF_BIRTH", "B-OTHER_INFORMATION", "I-OTHER_INFORMATION"]

# Convert token IDs back to tokens
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

# Convert predicted label IDs back to label names
predicted_labels = [label_list[p.item()] for p in predictions[0]]

# Group subword tokens and their labels
output = []
current_entity = []
current_label = None

for token, label in zip(tokens, predicted_labels):
    if token.startswith("##"):  # Handle subword tokens
        current_entity[-1] = current_entity[-1] + token[2:]  # Merge subword with previous token
    elif label != "O":  # If it's an entity label
        entity_type = label[2:]  # Remove the B- or I- prefix for the entity type
        if entity_type == current_label:  # Continue collecting tokens for the same entity
            current_entity.append(token)
        else:
            if current_entity:  # If there's a previous entity, finalize it
                output.append(f"{' '.join(current_entity)} ['{current_label}']")
            current_entity = [token]  # Start a new entity
            current_label = entity_type
    else:
        if current_entity:  # If we encounter "O" and have an ongoing entity, finalize it
            output.append(f"{' '.join(current_entity)} ['{current_label}']")
            current_entity = []
            current_label = None

# Handle the last entity if it exists
if current_entity:
    output.append(f"{' '.join(current_entity)} ['{current_label}']")

# Join the tokens into a sentence with spaces
output_text = " ".join(output)
print(output_text)


OSError: Incorrect path_or_model_id: './ner_model_new'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [ ]:
from transformers import BertTokenizerFast, BertForTokenClassification
import torch

# Load the fine-tuned model and tokenizer
model_dir = "./ner_model"  # Directory where the fine-tuned model is saved
tokenizer = BertTokenizerFast.from_pretrained(model_dir)
model = BertForTokenClassification.from_pretrained(model_dir)

# Set the model to evaluation mode
model.eval()

# Define the string to perform NER on
x = "Joshua is one of the most prominent figures and was born on 12-12-1990."

# Tokenize the text in variable x
inputs = tokenizer(x, return_tensors="pt", truncation=True, padding=True, is_split_into_words=False)

# Perform NER using the model
with torch.no_grad():
    outputs = model(**inputs)

# Get predictions (logits) and convert them into label IDs
predictions = torch.argmax(outputs.logits, dim=2)

# List of entity labels (ensure it's the same as used during fine-tuning)
label_list = ["O", "B-PERSON", "I-PERSON", "B-PERSON_ARABIC", "I-PERSON_ARABIC", "B-ALIAS", "I-ALIAS",
              "B-DATE_OF_BIRTH", "I-DATE_OF_BIRTH", "B-PLACE_OF_BIRTH", "I-PLACE_OF_BIRTH",
              "B-COUNTRY", "I-COUNTRY", "B-FUNCTION", "I-FUNCTION", "B-RANK", "I-RANK",
              "B-NATIONALITY", "I-NATIONALITY", "B-GENDER", "I-GENDER", "B-PASSPORT_NUMBER",
              "I-PASSPORT_NUMBER", "B-REASON_FOR_LISTING", "I-REASON_FOR_LISTING", "B-DATE_OF_LISTING",
              "I-DATE_OF_LISTING", "B-UN_LISTING_DATE", "I-UN_LISTING_DATE"]

# Convert token IDs back to tokens
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

# Convert predicted label IDs back to label names
predicted_labels = [label_list[p.item()] for p in predictions[0]]

# Clean up the output by handling special tokens and subword tokens
output = []
previous_token_is_subword = False

for token, label in zip(tokens, predicted_labels):
    # Skip special tokens like [CLS] and [SEP]
    if token in ["[CLS]", "[SEP]"]:
        continue

    # Join subword tokens with the previous token
    if token.startswith("##"):
        if previous_token_is_subword:
            output[-1] = f"{output[-1]}{token[2:]}"
        else:
            output[-1] = f"{output[-1]}{token[2:]}"
        previous_token_is_subword = True
    else:
        # Only include tokens with an entity label other than 'O'
        if label != "O":
            output.append(f"{token} ['{label}']")
        else:
            output.append(token)
        previous_token_is_subword = False

# Join the tokens into a sentence with spaces
output_text = " ".join(output)
print(output_text)


Joshua ['B-PERSON'] is ['I-REASON_FOR_LISTING'] one of ['I-REASON_FOR_LISTING'] the most prominent ['I-REASON_FOR_LISTING'] figures ['I-REASON_FOR_LISTING'] and was born on 12 ['B-DATE_OF_BIRTH'] - ['I-DATE_OF_BIRTH'] 12 ['I-DATE_OF_BIRTH'] - ['I-DATE_OF_BIRTH'] 1990 ['I-DATE_OF_BIRTH'] .


In [ ]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.3 MB/s eta 0:00:00


In [ ]:
import json
import faker

# Initialize Faker for generating random data
fake = faker.Faker()

# Dummy data you provided
dummy_data = [
    {"entry": "Nikolai Aleksandrovich LUKASHENKO (born on August 31, 2004) (also known as Mikalay Alyaksandravich LUKASHENKA and Kolya LUKASHENKO)"},
    {"entry": "Yuliya Aleksandrovna BLIZNIUK (born on September 23, 1971) (also known as Yuliya Aliaksandrauna BLIZNIUK, Julija Aleksandrovna BLIZNJUK and Julija Aljaksandrauna BLIZNJUK)"},
    {"entry": "Vitali Viktorovich SINILO (born on May 15, 1975) (also known as Vitaly Victorovich SINILA)"},
    {"entry": "Iryna Vasilyeuna PRADUN (born on May 21, 1974) (also known as Irina Vasilievna PRADUN)"},
    {"entry": "Valiantsina Piatrouna NOVIKAVA (born on October 12, 1978) (also known as Valentina Petrovna NOVIKOVA)"},
    {"entry": "Vadzim Ivanavich MAZOL (born on April 13, 1974) (also known as Vadim Ivanovich MOZOL)"},
    {"entry": "Aliaksandr Viktaravich ABASHYN (born on November 19, 1960)"},
    {"entry": "Oleg Grigorievich MISHCHENKO (born on August 11, 1968) (also known as Aleh Ryhoravich MISHCHANKA)"},
    {"entry": "Pavel Nikolaevich MURAVEIKO (born on September 26, 1971)"},
    {"entry": "Andrei Valentinovich FEDIN (born on April 8, 1971)"},
]

# Function to extract name, also known as, and date of birth from the entries
def extract_name_aka_dob(entry):
    name_part = entry.split("(born on")[0].strip()
    dob_part = entry.split("(born on")[1].split(")")[0].strip()

    if "(also known as" in entry:
        aka_part = entry.split("(also known as")[1].split(")")[0].strip()
    else:
        aka_part = None

    return {
        "name": name_part,
        "date_of_birth": dob_part,
        "also_known_as": aka_part
    }

# Extract data for each entry
extracted_data = [extract_name_aka_dob(item['entry']) for item in dummy_data]

# Save to JSON file
with open('dummy_extracted_dataset_name_aka_dob.json', 'w') as f:
    json.dump(extracted_data, f)

print("Dummy dataset saved successfully as 'dummy_extracted_dataset_name_aka_dob.json'")


Dummy dataset saved successfully as 'dummy_extracted_dataset_name_aka_dob.json'


In [ ]:
import json
import faker

# Initialize Faker for generating random data
fake = faker.Faker()

# Function to generate a custom record
def generate_custom_record():
    person_name = fake.name()
    aka = f"a) {fake.name()} b) {fake.name()}"  # Generating multiple alias names
    dob = fake.date_of_birth().strftime("%d %b %Y")  # Generating a random date of birth

    # Creating the text format
    text = f"Name: {person_name}, Also Known As: {aka}, Date of Birth: {dob}"

    # Entity structure
    entities = [
        {"start": text.find(person_name), "end": text.find(person_name) + len(person_name), "label": "NAME"},
        {"start": text.find(aka), "end": text.find(aka) + len(aka), "label": "ALSO_KNOWN_AS"},
        {"start": text.find(dob), "end": text.find(dob) + len(dob), "label": "DATE_OF_BIRTH"}
    ]

    return {"text": text, "entities": entities}

# Generate 5000 records
custom_records = [generate_custom_record() for _ in range(5000)]

# Save the records to a JSON file
file_path_custom = "custom_ner_dataset_name_aka_dob_5000.json"
with open(file_path_custom, 'w') as f:
    json.dump(custom_records, f)

print(f"Dataset saved to {file_path_custom}")


Dataset saved to custom_ner_dataset_name_aka_dob_5000.json


In [ ]:
from transformers import BertTokenizerFast, BertForTokenClassification
import torch

# Load the fine-tuned model and tokenizer
model_dir = "./ner_model"  # Directory where the fine-tuned model is saved
tokenizer = BertTokenizerFast.from_pretrained(model_dir)
model = BertForTokenClassification.from_pretrained(model_dir)

# Set the model to evaluation mode
model.eval()

# Define the string to perform NER on
x = "Joshua is one of the most prominent figures and was born on 12-12-1990."

# Tokenize the text in variable x
inputs = tokenizer(x, return_tensors="pt", truncation=True, padding=True, is_split_into_words=False)

# Perform NER using the model
with torch.no_grad():
    outputs = model(**inputs)

# Get predictions (logits) and convert them into label IDs
predictions = torch.argmax(outputs.logits, dim=2)

# List of entity labels (ensure it's the same as used during fine-tuning)
label_list = ["O", "B-PERSON", "I-PERSON", "B-PERSON_ARABIC", "I-PERSON_ARABIC", "B-ALIAS", "I-ALIAS",
              "B-DATE_OF_BIRTH", "I-DATE_OF_BIRTH", "B-PLACE_OF_BIRTH", "I-PLACE_OF_BIRTH",
              "B-COUNTRY", "I-COUNTRY", "B-FUNCTION", "I-FUNCTION", "B-RANK", "I-RANK",
              "B-NATIONALITY", "I-NATIONALITY", "B-GENDER", "I-GENDER", "B-PASSPORT_NUMBER",
              "I-PASSPORT_NUMBER", "B-REASON_FOR_LISTING", "I-REASON_FOR_LISTING", "B-DATE_OF_LISTING",
              "I-DATE_OF_LISTING", "B-UN_LISTING_DATE", "I-UN_LISTING_DATE"]

# Convert token IDs back to tokens
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

# Convert predicted label IDs back to label names
predicted_labels = [label_list[p.item()] for p in predictions[0]]

# Clean up the output by handling special tokens and subword tokens
output = []
previous_label = "O"
previous_token_is_subword = False

for token, label in zip(tokens, predicted_labels):
    # Skip special tokens like [CLS] and [SEP]
    if token in ["[CLS]", "[SEP]"]:
        continue

    # Join subword tokens with the previous token
    if token.startswith("##"):
        if previous_token_is_subword:
            output[-1] = f"{output[-1]}{token[2:]}"
        else:
            output[-1] = f"{output[-1]}{token[2:]}"
        previous_token_is_subword = True
    else:
        # Only include tokens with a relevant entity label
        if label.startswith("B-") or (label.startswith("I-") and previous_label != "O"):
            output.append(f"{token} ['{label}']")
        else:
            output.append(token)
        previous_label = label
        previous_token_is_subword = False

# Join the tokens into a sentence with spaces
output_text = " ".join(output)
print(output_text)


Joshua ['B-PERSON'] is ['I-REASON_FOR_LISTING'] one of the most prominent figures ['I-REASON_FOR_LISTING'] and was born on 12 ['B-DATE_OF_BIRTH'] - ['I-DATE_OF_BIRTH'] 12 ['I-DATE_OF_BIRTH'] - ['I-DATE_OF_BIRTH'] 1990 ['I-DATE_OF_BIRTH'] .


In [ ]:
from transformers import BertTokenizerFast, BertForTokenClassification
import torch

# Load the fine-tuned model and tokenizer
model_dir = "./ner_model"  # Directory where the fine-tuned model is saved
tokenizer = BertTokenizerFast.from_pretrained(model_dir)
model = BertForTokenClassification.from_pretrained(model_dir)

# Set the model to evaluation mode
model.eval()

# Define the string to perform NER on
x = """7.    Name: 1: ABU 2: BAKR 3: YUNIS 4: JABIR

Title: Major General Designation: Defence Minister. DOB: 1952 POB: Jalo, Libya Good quality a.k.a.: na Low quality a.k.a.: na Nationality: na Passport no: na National identification no: na Address: na Listed on: 26 Feb. 2011 (amended on 2 Apr. 2012) Other information: Listed pursuant to paragraph 15 of resolution 1970 (Travel Ban). Listed on 17 March 2011 pursuant to paragraph 17 of resolution 1970 (Asset Freeze). Believed status/location: deceased. INTERPOL-UN Security Council Special Notice web link: https://www.interpol.int/en/notice/search/un/5525775

"""

# Tokenize the text in variable x
inputs = tokenizer(x, return_tensors="pt", truncation=True, padding=True, is_split_into_words=False)

# Perform NER using the model
with torch.no_grad():
    outputs = model(**inputs)

# Get predictions (logits) and convert them into label IDs
predictions = torch.argmax(outputs.logits, dim=2)

# List of entity labels (ensure it's the same as used during fine-tuning)
label_list = ["O", "B-PERSON", "I-PERSON", "B-PERSON_ARABIC", "I-PERSON_ARABIC", "B-ALIAS", "I-ALIAS",
              "B-DATE_OF_BIRTH", "I-DATE_OF_BIRTH", "B-PLACE_OF_BIRTH", "I-PLACE_OF_BIRTH",
              "B-COUNTRY", "I-COUNTRY", "B-FUNCTION", "I-FUNCTION", "B-RANK", "I-RANK",
              "B-NATIONALITY", "I-NATIONALITY", "B-GENDER", "I-GENDER", "B-PASSPORT_NUMBER",
              "I-PASSPORT_NUMBER", "B-REASON_FOR_LISTING", "I-REASON_FOR_LISTING", "B-DATE_OF_LISTING",
              "I-DATE_OF_LISTING", "B-UN_LISTING_DATE", "I-UN_LISTING_DATE"]

# Convert token IDs back to tokens
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

# Convert predicted label IDs back to label names
predicted_labels = [label_list[p.item()] for p in predictions[0]]

# Clean up the output by handling special tokens and subword tokens
output = []
previous_label = "O"
previous_token_is_subword = False

for token, label in zip(tokens, predicted_labels):
    # Skip special tokens like [CLS] and [SEP]
    if token in ["[CLS]", "[SEP]"]:
        continue

    # Join subword tokens with the previous token
    if token.startswith("##"):
        if previous_token_is_subword:
            output[-1] = f"{output[-1]}{token[2:]}"
        else:
            output[-1] = f"{output[-1]}{token[2:]}"
        previous_token_is_subword = True
    else:
        # Only include tokens with a relevant entity label
        if label.startswith("B-"):
            output.append(f"{token} ['{label}']")
            previous_label = label
        elif label.startswith("I-") and previous_label[2:] == label[2:]:
            # Only include 'I-' labels if they follow the correct 'B-' label
            output.append(f"{token} ['{label}']")
        else:
            output.append(token)  # Normal token without entity
            previous_label = "O"
        previous_token_is_subword = False

# Join the tokens into a sentence with spaces
output_text = " ".join(output)
print(output_text)


7 . Name : 1 : ABU 2 : BAKR 3 : YUNIS 4 : JABIR Title : Major ['B-RANK'] General Designation : Defence ['B-FUNCTION'] Minister ['I-FUNCTION'] . DOB : 1952 ['B-PLACE_OF_BIRTH'] POB : J ['B-PLACE_OF_BIRTH']alo , Libya ['B-COUNTRY'] Good quality a . k . a . : na ['B-PASSPORT_NUMBER'] Low quality a . k . a . : na ['B-PASSPORT_NUMBER'] Nationality : na Passport no : na National identification no : na Address : na Listed on : 26 ['B-DATE_OF_LISTING'] Feb ['I-DATE_OF_LISTING'] . ['I-DATE_OF_LISTING'] 2011 ['I-DATE_OF_LISTING'] ( amended on 2 ['B-DATE_OF_LISTING'] Apr ['I-DATE_OF_LISTING'] . ['I-DATE_OF_LISTING'] 2012 ['I-DATE_OF_LISTING'] ) Other information : Listed pursuant to paragraph 15 of resolution 1970 ( Travel Ban ) . Listed on 17 ['B-DATE_OF_LISTING'] March ['I-DATE_OF_LISTING'] 2011 ['I-DATE_OF_LISTING'] pursuant to paragraph 17 of resolution 1970 ( Asset Freeze ) . Believed status / location : deceased . INTERPOL - UN Security Council Special Notice web link : https : / / www . in

In [ ]:
import re
from bs4 import BeautifulSoup

def clean_text(text):
    # Remove extra whitespace and newlines
    return re.sub(r'\s+', ' ', text).strip()

def extract_table_data(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    tables = soup.find_all('table')

    all_data = []

    for table in tables:
        headers = [clean_text(th.text) for th in table.find_all('th')]

        if not headers:
            # If no th elements, use first row as headers
            headers = [clean_text(td.text) for td in table.find('tr').find_all('td')]

        rows = table.find_all('tr')[1:]  # Skip the header row

        for row in rows:
            cells = row.find_all('td')
            if len(cells) == len(headers):
                entry = {}
                for header, cell in zip(headers, cells):
                    entry[header] = clean_text(cell.text)
                all_data.append(entry)

    return all_data

# Example usage
html_content = """
<div id="anx_III">
         <p>
            <br>
         </p>
         <hr class="separator-annex">
         <p>
            <br>
         </p>
         <p class="title-annex-1" id="id-9b29d3e7-f299-4373-a9b6-7b45e83fd694">ANNEX III</p>
         <p class="title-gr-seq-level-1" id="id-8cdc45e9-a37f-4a1d-82b2-040fc32dd504">
            <span class="boldface">LIST OF NATURAL AND LEGAL PERSONS, ENTITIES OR BODIES REFERRED TO IN ARTICLE 6(2)</span>
         </p>
         <p class="title-gr-seq-level-2" id="id-079c13d1-4976-4af5-b20a-b6a5d62a6193">A.&nbsp;&nbsp;&nbsp;<span class="boldface">Persons</span>
         </p>
         <p>
            <br>
         </p>
         <div class="centered">
            <p>
               <br>
            </p>
            <table width="960" border="1" cellpadding="2" cellspacing="0" class="borderOj">
               <colgroup><col width="0%">
               <col width="11%">
               <col width="25%">
               <col width="63%">
               <col width="1%">
               </colgroup><tbody>
                  <tr>
                     <td style="font-weight: bold" valign="top">
                        <p class="normal">&nbsp;</p>
                        <div>
                     </div></td>
                     <td style="font-weight: bold" valign="top">
                        <p class="tbl-norm">Name</p>
                     </td>
                     <td style="font-weight: bold" valign="top">
                        <p class="tbl-norm">Identifying information</p>
                     </td>
                     <td style="font-weight: bold" valign="top">
                        <p class="tbl-norm">Reasons</p>
                     </td>
                     <td style="font-weight: bold" valign="top">
                        <p class="tbl-norm">Date of listing</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32019R1292" onclick="window.open(this.href,'_blank'); return false;" title="32019R1292: DELETED">▼M27</a><a class="anchorarrow" id="M27-1" href="#M27-2"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>&nbsp;—————</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32016R0044" onclick="window.open(this.href,'_blank'); return false;" title="32016R0044">▼B</a><a class="anchorarrow" id="B-13" href="#B-14"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>
                        </p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">
                           <span>
                              <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32019R1292" onclick="window.open(this.href,'_blank'); return false;" title="32019R1292: REPLACED">
                                 <span class="boldface">►M27</span>
                              </a><a class="anchorarrow" id="M27-2" href="#M27-3"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>
                           </span>&nbsp;1.<span class="boldface">&nbsp;◄ </span>
                        </p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">ABU SHAARIYA</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Position: Deputy Head, External Security Organisation</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Brother-in-law of Muammar Qadhafi.</p>
                        <p class="tbl-norm">Prominent member of the Qadhafi regime and as such closely associated with the former regime of Muammar Qadhafi.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">28.2.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32018R1073" onclick="window.open(this.href,'_blank'); return false;" title="32018R1073: DELETED">▼M21</a><a class="anchorarrow" id="M21-1" href="#M21-2"><i class="fa fa-circle" title="LAST" style="text-indent: 5pt;" aria-hidden="true"></i></a>&nbsp;—————</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32016R0044" onclick="window.open(this.href,'_blank'); return false;" title="32016R0044">▼B</a><a class="anchorarrow" id="B-14" href="#B-15"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>
                        </p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">
                           <span>
                              <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32019R1292" onclick="window.open(this.href,'_blank'); return false;" title="32019R1292: REPLACED">
                                 <span class="boldface">►M27</span>
                              </a><a class="anchorarrow" id="M27-3" href="#M27-4"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>
                           </span>&nbsp;2.<span class="boldface">&nbsp;◄ </span>
                        </p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">ALSHARGAWI, Bashir Saleh Bashir</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Date of birth: 1946</p>
                        <p class="tbl-norm">Place of birth: Traghen</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Head of Cabinet of Muammar Qadhafi. Closely associated with the former regime of Muammar Qadhafi.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">28.2.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32021R1241" onclick="window.open(this.href,'_blank'); return false;" title="32021R1241: DELETED">▼M36</a><a class="anchorarrow" id="M36-1" href="#M36-2"><i class="fa fa-circle" title="LAST" style="text-indent: 5pt;" aria-hidden="true"></i></a>&nbsp;—————</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32016R0044" onclick="window.open(this.href,'_blank'); return false;" title="32016R0044">▼B</a><a class="anchorarrow" id="B-15" href="#B-16"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>
                        </p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">
                           <span>
                              <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32019R1292" onclick="window.open(this.href,'_blank'); return false;" title="32019R1292: REPLACED">
                                 <span class="boldface">►M27</span>
                              </a><a class="anchorarrow" id="M27-4" href="#M27-5"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>
                           </span>&nbsp;4.<span class="boldface">&nbsp;◄ </span>
                        </p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">FARKASH, Mohammed Boucharaya</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Date of birth: 1 July 1949</p>
                        <p class="tbl-norm">Place of birth: Al-Bayda</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Former director of intelligence in External Security Office.</p>
                        <p class="tbl-norm">Closely associated with the former regime of Muammar Qadhafi.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">28.2.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">
                           <span>
                              <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32019R1292" onclick="window.open(this.href,'_blank'); return false;" title="32019R1292: REPLACED">
                                 <span class="boldface">►M27</span>
                              </a><a class="anchorarrow" id="M27-5" href="#M27-6"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>
                           </span>&nbsp;5.<span class="boldface">&nbsp;◄ </span>
                        </p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">EL-KASSIM ZOUAI, Mohamed Abou</p>
                     </td>
                     <td valign="top">
                        <p class="normal">&nbsp;</p>
                        <div>
                     </div></td>
                     <td valign="top">
                        <p class="tbl-norm">Former Secretary General of the General People's Congress.</p>
                        <p class="tbl-norm">Closely associated with the former regime of Muammar Qadhafi.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">21.3.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32021R1932" onclick="window.open(this.href,'_blank'); return false;" title="32021R1932: DELETED">▼M38</a><a class="anchorarrow" id="M38-1" href="#M38-2"><i class="fa fa-circle" title="LAST" style="text-indent: 5pt;" aria-hidden="true"></i></a>&nbsp;—————</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32016R0044" onclick="window.open(this.href,'_blank'); return false;" title="32016R0044">▼B</a><a class="anchorarrow" id="B-16" href="#B-17"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>
                        </p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">
                           <span>
                              <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32019R1292" onclick="window.open(this.href,'_blank'); return false;" title="32019R1292: REPLACED">
                                 <span class="boldface">►M27</span>
                              </a><a class="anchorarrow" id="M27-6" href="#M27-7"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>
                           </span>&nbsp;7.<span class="boldface">&nbsp;◄ </span>
                        </p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">HIJAZI, Mohamad Mahmoud</p>
                     </td>
                     <td valign="top">
                        <p class="normal">&nbsp;</p>
                        <div>
                     </div></td>
                     <td valign="top">
                        <p class="tbl-norm">Minister for Health and Environment in Colonel Qadhafi's Government.</p>
                        <p class="tbl-norm">Closely associated with the former regime of Muammar Qadhafi.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">21.3.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32023R2501" onclick="window.open(this.href,'_blank'); return false;" title="32023R2501: REPLACED">▼M47</a><a class="anchorarrow" id="M47-1" href="#M47-2"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>
                        </p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">8.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">HOUEJ, Mohamad Ali</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Minister of Economy and Trade in the Government of National Unity of Libya</p>
                        <p class="tbl-norm">Date of birth: 1949</p>
                        <p class="tbl-norm">Place of birth: Al-Azizia (near Tripoli)</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Minister for Industry, Economy and Trade in Colonel Qadhafi’s Government.</p>
                        <p class="tbl-norm">Closely associated with the former regime of Muammar Qadhafi.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">21.3.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32021R0667" onclick="window.open(this.href,'_blank'); return false;" title="32021R0667: DELETED">▼M34</a><a class="anchorarrow" id="M34-1" href="#M34-2"><i class="fa fa-circle" title="LAST" style="text-indent: 5pt;" aria-hidden="true"></i></a>&nbsp;—————</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32016R0044" onclick="window.open(this.href,'_blank'); return false;" title="32016R0044">▼B</a><a class="anchorarrow" id="B-17" href="#B-18"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>
                        </p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">
                           <span>
                              <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32019R1292" onclick="window.open(this.href,'_blank'); return false;" title="32019R1292: REPLACED">
                                 <span class="boldface">►M27</span>
                              </a><a class="anchorarrow" id="M27-7" href="#M27-8"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>
                           </span>&nbsp;10.<span class="boldface">&nbsp;◄ </span>
                        </p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">AL-CHARIF, Ibrahim Zarroug</p>
                     </td>
                     <td valign="top">
                        <p class="normal">&nbsp;</p>
                        <div>
                     </div></td>
                     <td valign="top">
                        <p class="tbl-norm">Minister for Social Affairs in Colonel Qadhafi's Government.</p>
                        <p class="tbl-norm">Closely associated with the former regime of Muammar Qadhafi.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">21.3.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">
                           <span>
                              <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32019R1292" onclick="window.open(this.href,'_blank'); return false;" title="32019R1292: REPLACED">
                                 <span class="boldface">►M27</span>
                              </a><a class="anchorarrow" id="M27-8" href="#M27-9"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>
                           </span>&nbsp;11.<span class="boldface">&nbsp;◄ </span>
                        </p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">FAKHIRI, Abdelkebir Mohamad</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Date of birth: 4 May 1963</p>
                        <p class="tbl-norm">Passport number: B/014965 (expired end 2013)</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Minister for Education, Higher Education and Research in Colonel Qadhafi's Government. Closely associated with the former regime of Muammar Qadhafi.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">21.3.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32023R2501" onclick="window.open(this.href,'_blank'); return false;" title="32023R2501: DELETED">▼M47</a><a class="anchorarrow" id="M47-2" href="#M47-3"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>&nbsp;—————</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32016R1687" onclick="window.open(this.href,'_blank'); return false;" title="32016R1687: DELETED">▼M5</a><a class="anchorarrow" id="M5-1" href="#M5-2"><i class="fa fa-circle" title="LAST" style="text-indent: 5pt;" aria-hidden="true"></i></a>&nbsp;—————</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32019R1292" onclick="window.open(this.href,'_blank'); return false;" title="32019R1292: DELETED">▼M27</a><a class="anchorarrow" id="M27-9" href="#M27-10"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>&nbsp;—————</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32016R0044" onclick="window.open(this.href,'_blank'); return false;" title="32016R0044">▼B</a><a class="anchorarrow" id="B-18" href="#B-19"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>
                        </p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">
                           <span>
                              <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32019R1292" onclick="window.open(this.href,'_blank'); return false;" title="32019R1292: REPLACED">
                                 <span class="boldface">►M27</span>
                              </a><a class="anchorarrow" id="M27-10" href="#M27-11"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>
                           </span>&nbsp;13.<span class="boldface">&nbsp;◄ </span>
                        </p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">DIBRI, Abdulqader Yusef</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Position: Head of Muammar Qadhafi's personal security</p>
                        <p class="tbl-norm">Date of Birth: 1946</p>
                        <p class="tbl-norm">Place of Birth: Houn, Libya</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Responsibility for regime security. History of directing violence against dissidents.</p>
                        <p class="tbl-norm">Closely associated with the former regime of Muammar Qadhafi.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">28.2.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32023R2501" onclick="window.open(this.href,'_blank'); return false;" title="32023R2501: DELETED">▼M47</a><a class="anchorarrow" id="M47-3" href="#M47-4"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>&nbsp;—————</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32022R1308" onclick="window.open(this.href,'_blank'); return false;" title="32022R1308: REPLACED">▼M42</a><a class="anchorarrow" id="M42-1" href="#M42-2"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>
                        </p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">15.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">AL QADHAFI, Quren Salih Quren</p>
                        <p class="tbl-norm">a.k.a. Akrin Akrin Saleh, Al Qadhafi Qurayn Salih Qurayn, Al&nbsp;Qadhafi Qu’ren Salih Qu’ren, Salah Egreen</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Gender: male</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Former Libyan Ambassador to Chad. Has left Chad for Sabha. Involved directly in recruiting and coordinating mercenaries for the regime of the late Muammar Qadhafi.</p>
                        <p class="tbl-norm">Closely associated with the former regime of Muammar Qadhafi.</p>
                        <p class="tbl-norm">Member of the Popular Front for the Liberation of Libya (PFLL), a militia and political party loyal to the late Muammar Qadhafi. Involved in undermining the successful completion of Libya’s political transition by opposing the UN and undermining the UN-facilitated political process, including the Libyan Political Dialogue Forum (LPDF), thereby posing a continued risk to the peace, stability and security of Libya.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">12.4.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32016R0044" onclick="window.open(this.href,'_blank'); return false;" title="32016R0044">▼B</a><a class="anchorarrow" id="B-19" href="#B-20"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>
                        </p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">
                           <span>
                              <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32019R1292" onclick="window.open(this.href,'_blank'); return false;" title="32019R1292: REPLACED">
                                 <span class="boldface">►M27</span>
                              </a><a class="anchorarrow" id="M27-11" href="#M27-12"><i class="fa fa-circle" title="LAST" style="text-indent: 5pt;" aria-hidden="true"></i></a>
                           </span>&nbsp;16.<span class="boldface">&nbsp;◄ </span>
                        </p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">AL KUNI, Colonel Amid Husain</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Believed status/location: South Libya</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Former Governor of Ghat (South Libya). Directly involved in recruiting mercenaries.</p>
                        <p class="tbl-norm">Closely associated with the former regime of Muammar Qadhafi.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">12.4.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32020R1380" onclick="window.open(this.href,'_blank'); return false;" title="32020R1380: DELETED">▼M31</a><a class="anchorarrow" id="M31-1" href="#M31-2"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>&nbsp;—————</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32021R0538" onclick="window.open(this.href,'_blank'); return false;" title="32021R0538: DELETED">▼M33</a><a class="anchorarrow" id="M33-1" href="#M33-2"><i class="fa fa-circle" title="LAST" style="text-indent: 5pt;" aria-hidden="true"></i></a>&nbsp;—————</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32020R1380" onclick="window.open(this.href,'_blank'); return false;" title="32020R1380: DELETED">▼M31</a><a class="anchorarrow" id="M31-2" href="#M31-3"><i class="fa fa-circle" title="LAST" style="text-indent: 5pt;" aria-hidden="true"></i></a>&nbsp;—————</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32022R1308" onclick="window.open(this.href,'_blank'); return false;" title="32022R1308: DELETED">▼M42</a><a class="anchorarrow" id="M42-2" href="#M42-3"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>&nbsp;—————</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32020R1309" onclick="window.open(this.href,'_blank'); return false;" title="32020R1309: INSERTED">▼M30</a><a class="anchorarrow" id="M30-1" href="#M30-2"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>
                        </p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">21.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">DIAB, Moussa</p>
                        <p class="tbl-norm">a.k.a. DIAB, Mousa</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Gender: male</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Moussa Diab is responsible for and has directly engaged in serious human rights abuses including human trafficking and the kidnapping, raping and killing of migrants and refugees.</p>
                        <p class="tbl-norm">He held migrants and refugees in captivity in an illegal detention camp near Bani Walid, where they were treated in an inhuman and degrading manner. Several migrants and refugees were killed when they tried to escape from the detention camp.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">21.9.2020</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32022R1308" onclick="window.open(this.href,'_blank'); return false;" title="32022R1308: REPLACED">▼M42</a><a class="anchorarrow" id="M42-3" href="#M42-4"><i class="fa fa-circle" title="LAST" style="text-indent: 5pt;" aria-hidden="true"></i></a>
                        </p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">22.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Yevgeniy Viktorovich PRIGOZHIN</p>
                        <p class="tbl-norm">(Евгений Викторович Пригожин)</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Date of birth: 1&nbsp;June 1961</p>
                        <p class="tbl-norm">Place of birth: Leningrad, former USSR (now St.&nbsp;Petersburg, Russian Federation)</p>
                        <p class="tbl-norm">Nationality: Russian</p>
                        <p class="tbl-norm">Gender: male</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Yevgeniy Viktorovich Prigozhin is a Russian businessman with close links, including financial, to the Wagner Group, a Russia-based unincorporated private military entity.</p>
                        <p class="tbl-norm">In this way, Prigozhin is engaged in and provides support for the Wagner Group’s activities in Libya, which threaten the country’s peace, stability and security.</p>
                        <p class="tbl-norm">In particular, the Wagner Group is involved in multiple and repeated breaches of the arms embargo in Libya established in UNSCR&nbsp;1970 (2011) and transposed in Article&nbsp;1 of Decision (CFSP)&nbsp;2015/1333, including the delivery of arms and the deployment of mercenaries into Libya in support of the Libya National Army. The Wagner Group has participated in multiple military operations against the UN-endorsed Government of National Accord and has contributed to damaging the stability of Libya and undermining a peaceful process.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">15.10.2020</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32021R2192" onclick="window.open(this.href,'_blank'); return false;" title="32021R2192: INSERTED">▼M39</a><a class="anchorarrow" id="M39-1" href="#M39-2"><i class="fa fa-circle" title="LAST" style="text-indent: 5pt;" aria-hidden="true"></i></a>
                        </p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">23.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">KUZNETSOV, Aleksandr (Alexander) Sergeevich</p>
                        <p class="tbl-norm">(Александр Сергеевич КУЗНЕЦОВ)</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Position (s): Commander of&nbsp;1st Attack and Reconnaissance Company of the Wagner Group</p>
                        <p class="tbl-norm">Callsign: Ratibor</p>
                        <p class="tbl-norm">Date of birth: 8&nbsp;October&nbsp;1977</p>
                        <p class="tbl-norm">Place of birth: Nikolskoye, former USSR (now Russian Federation)</p>
                        <p class="tbl-norm">Nationality: Russian</p>
                        <p class="tbl-norm">Wagner Group ID: M-0271</p>
                        <p class="tbl-norm">Gender: male</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Aleksandr Sergeevich Kuznetsov belongs to the command structure of the Wagner Group, a Russia-based unincorporated private military entity. He took command of the&nbsp;1st Attack and Reconnaissance Company of the Wagner Group in&nbsp;2014. He was injured in&nbsp;September&nbsp;2019 in Libya while fighting alongside the Libyan National Liberation Army (NLA) as commander of the Wagner Group’s military operatives. Through his position and actions, Kuznetsov is responsible for the Wagner Group’s activities that threaten the peace, stability and security of Libya.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">13.12.2021</p>
                     </td>
                  </tr>
               </tbody>
            </table>
         </div>
         <p class="modref">
            <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32016R0044R%2801%29" onclick="window.open(this.href,'_blank'); return false;" title="32016R0044R(01): REPLACED">▼C5</a><a class="anchorarrow" id="C5-1" href="#C5-2"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>
         </p>
         <p class="title-gr-seq-level-2" id="id-78988296-a324-4b13-88cc-02a1c6d99b30">B.&nbsp;&nbsp;&nbsp;<span class="boldface">Entities</span>
         </p>
         <p>
            <br>
         </p>
         <div class="centered">
            <p>
               <br>
            </p>
            <table width="960" border="1" cellpadding="2" cellspacing="0" class="borderOj">
               <colgroup><col width="0%">
               <col width="5%">
               <col width="64%">
               <col width="29%">
               <col width="1%">
               </colgroup><tbody>
                  <tr>
                     <td style="font-weight: bold" valign="top">
                        <p class="normal">&nbsp;</p>
                        <div>
                     </div></td>
                     <td style="font-weight: bold" valign="top">
                        <p class="tbl-norm">Name</p>
                     </td>
                     <td style="font-weight: bold" valign="top">
                        <p class="tbl-norm">Identifying information</p>
                     </td>
                     <td style="font-weight: bold" valign="top">
                        <p class="tbl-norm">Reasons</p>
                     </td>
                     <td style="font-weight: bold" valign="top">
                        <p class="tbl-norm">Date of listing</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32022R2525" onclick="window.open(this.href,'_blank'); return false;" title="32022R2525: DELETED">▼M44</a><a class="anchorarrow" id="M44-1" href="#M44-2"><i class="fa fa-circle" title="LAST" style="text-indent: 5pt;" aria-hidden="true"></i></a>&nbsp;—————</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32016R0044R%2801%29" onclick="window.open(this.href,'_blank'); return false;" title="32016R0044R(01): REPLACED">▼C5</a><a class="anchorarrow" id="C5-2" href="#C5-3"><i class="fa fa-circle" title="LAST" style="text-indent: 5pt;" aria-hidden="true"></i></a>
                        </p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">2.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Gaddafi International Charity and Development Foundation</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Contact details of administration: Hay Alandalus — Jian St. — Tripoli — PoBox: 1101 — LIBYA Telephone: (+ 218) 214778301 — Fax (+ 218) 214778766; email: info@gicdf.org</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Closely associated with the former regime of Muammar Qadhafi.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">21.3.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">3.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Waatassimou Foundation</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Based in Tripoli.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Closely associated with the former regime of Muammar Qadhafi.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">21.3.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">4.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Libyan Jamahirya Broadcasting Corporation</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Contact details: tel. 00 218 21 444 59 26; 00 21 444 59 00; fax 00 218 21 340 21 07 http://www.ljbc.net; email: info@ljbc.net</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Closely associated with the former regime of Muammar Qadhafi.</p>
                        <p class="tbl-norm">Involved in public incitement to hatred and violence through participation in disinformation campaigns concerning violence against demonstrators.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">21.3.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">5.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Revolutionary Guard Corps</p>
                     </td>
                     <td valign="top">
                        <p class="normal">&nbsp;</p>
                        <div>
                     </div></td>
                     <td valign="top">
                        <p class="tbl-norm">Closely associated with the former regime of Muammar Qadhafi.</p>
                        <p class="tbl-norm">Involved in violence against demonstrators.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">21.3.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">6.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Libyan Agricultural Bank (a.k.a. Agricultural Bank; a.k.a. Al Masraf Al Zirae Agricultural Bank; a.k.a. Al Masraf Al Zirae; a.k.a. Libyan Agricultural Bank)</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">El Ghayran Area, Ganzor El Sharqya, P.O. Box 1100, Tripoli, Libya; Al Jumhouria Street, East Junzour, Al Gheran, Tripoli, Libya; Email Address agbank@agribankly.org; SWIFT/BIC AGRULYLT (Libya);</p>
                        <p class="tbl-norm">Tel. No. (218)214870586;</p>
                        <p class="tbl-norm">Tel. No. (218) 214870714;</p>
                        <p class="tbl-norm">Tel. No. (218) 214870745;</p>
                        <p class="tbl-norm">Tel. No. (218) 213338366;</p>
                        <p class="tbl-norm">Tel. No. (218) 213331533;</p>
                        <p class="tbl-norm">Tel. No. (218) 213333541;</p>
                        <p class="tbl-norm">Tel. No. (218) 213333544;</p>
                        <p class="tbl-norm">Tel. No. (218) 213333543;</p>
                        <p class="tbl-norm">Tel. No. (218) 213333542;</p>
                        <p class="tbl-norm">Fax No. (218) 214870747;</p>
                        <p class="tbl-norm">Fax No. (218) 214870767;</p>
                        <p class="tbl-norm">Fax No. (218) 214870777;</p>
                        <p class="tbl-norm">Fax No. (218) 213330927;</p>
                        <p class="tbl-norm">Fax No. (218) 213333545</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Libyan subsidiary of the Central Bank of Libya.</p>
                        <p class="tbl-norm">Closely associated with the former regime of Muammar Qadhafi.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">12.4.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">7.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Al-Inma Holding Co. for Services Investments</p>
                     </td>
                     <td valign="top">
                        <p class="normal">&nbsp;</p>
                        <div>
                     </div></td>
                     <td valign="top">
                        <p class="tbl-norm">Libyan subsidiary of the Economic &amp; Social Development Fund.</p>
                        <p class="tbl-norm">Closely associated with the former regime of Muammar Qadhafi.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">12.4.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">8.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Al-Inma Holding Co. For Industrial Investments</p>
                     </td>
                     <td valign="top">
                        <p class="normal">&nbsp;</p>
                        <div>
                     </div></td>
                     <td valign="top">
                        <p class="tbl-norm">Libyan subsidiary of the Economic &amp; Social Development Fund.</p>
                        <p class="tbl-norm">Closely associated with the former regime of Muammar Qadhafi.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">12.4.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">9.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Al-Inma Holding Company for Tourism Investment</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Hasan al-Mashay Street (off al- Zawiyah Street) Tel.&nbsp;No.:&nbsp;(218)&nbsp;213345187 Fax +218.21.334.5188 email:&nbsp;info@ethic.ly</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Libyan subsidiary of the Economic &amp; Social Development Fund.</p>
                        <p class="tbl-norm">Closely associated with the former regime of Muammar Qadhafi.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">12.4.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">10.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Al-Inma Holding Co. for Construction and Real Estate Developments</p>
                     </td>
                     <td valign="top">
                        <p class="normal">&nbsp;</p>
                        <div>
                     </div></td>
                     <td valign="top">
                        <p class="tbl-norm">Libyan subsidiary of the Economic &amp; Social Development Fund.</p>
                        <p class="tbl-norm">Closely associated with the former regime of Muammar Qadhafi.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">12.4.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">11.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">LAP Green Networks (a.k.a. Lap GreenN, LAP Green Holding Company)</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">9th Floor, Ebene Tower, 52, Cybercity, Ebene, Mauritius</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Libyan subsidiary of the Libyan Africa Investment Portfolio.</p>
                        <p class="tbl-norm">Closely associated with the former regime of Muammar Qadhafi.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">12.4.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">12.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Sabtina Ltd</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">530-532 Elder Gate, Elder House, Milton Keynes, UK</p>
                        <p class="tbl-norm">Other info: Reg no 01794877 (UK)</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">UK-incorporated subsidiary of the Libyan Investment Authority.</p>
                        <p class="tbl-norm">Closely associated with the former regime of Muammar Qadhafi.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">12.4.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">13.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Ashton Global Investments Limited</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Woodbourne Hall, PO Box 3162, Road Town, Tortola, British Virgin Islands</p>
                        <p class="tbl-norm">Other Info: Reg no 1510484 (BVI)</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">BVI-incorporated subsidiary of the Libyan Investment Authority.</p>
                        <p class="tbl-norm">Closely associated with the former regime of Muammar Qadhafi.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">12.4.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">14.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Capitana Seas Limited</p>
                     </td>
                     <td valign="top">
                        <p class="normal">&nbsp;</p>
                        <div>
                     </div></td>
                     <td valign="top">
                        <p class="tbl-norm">BVI-incorporated entity owned by Saadi Qadhafi</p>
                        <p class="tbl-norm">Closely associated with the former regime of Muammar Qadhafi.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">12.4.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">15.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Kinloss Property Limited</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Woodbourne Hall, PO Box 3162, Road Town, Tortola, British Virgin Islands</p>
                        <p class="tbl-norm">Other Info: Reg no 1534407 (BVI)</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">BVI-incorporated subsidiary of the Libyan Investment Authority.</p>
                        <p class="tbl-norm">Closely associated with the former regime of Muammar Qadhafi.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">12.4.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32023R2501" onclick="window.open(this.href,'_blank'); return false;" title="32023R2501: DELETED">▼M47</a><a class="anchorarrow" id="M47-4" href="#M47-5"><i class="fa fa-circle" title="LAST" style="text-indent: 5pt;" aria-hidden="true"></i></a>&nbsp;—————</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32020R1309" onclick="window.open(this.href,'_blank'); return false;" title="32020R1309: INSERTED">▼M30</a><a class="anchorarrow" id="M30-2" href="#M30-3"><i class="fa fa-circle" title="LAST" style="text-indent: 5pt;" aria-hidden="true"></i></a>
                        </p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">17.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Sigma Airlines</p>
                        <p class="tbl-norm">a.k.a. Sigma Aviation; Air Sigma</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Address:</p>
                        <p class="normal">Markov Str. 11</p>
                        <p class="normal">050013, Almaty, Kazakhstan</p>
                        <p class="tbl-norm">Tel.: +77272922305</p>
                        <p class="tbl-norm">Website: https://airsigma.pro/</p>
                        <p class="tbl-norm">Registered under name: Kenesbayev Umirbek Zharmenovich</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Sigma Airlines is a commercial cargo air company which operates aircrafts found to have violated the arms embargo in Libya established in UNSCR 1970 (2011) and transposed in Article 1 of Decision (CFSP) 2015/1333.</p>
                        <p class="tbl-norm">The UN has identified Sigma Airlines as one of the commercial air cargo providers operating in non-compliance with the UN embargo on the transfer of military material to Libya.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">21.9.2020</p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">18.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Avrasya Shipping</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Address:</p>
                        <p class="tbl-norm">Liman Mh. Gezi Cd. No:22/3 İlkadım, Samsun, Turkey</p>
                        <p class="tbl-norm">Tel.: +90 549 720 1748</p>
                        <p class="tbl-norm">E-mail: info@avrasyashipping.com</p>
                        <p class="tbl-norm">Website: http://www.avrasyashipping.com/iletisim</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Avrasya Shipping is a maritime company which operates a vessel called Cirkin found to have violated the arms embargo in Libya established in UNSCR 1970 (2011) and transposed in Article 1 of Decision (CFSP) 2015/1333.</p>
                        <p class="tbl-norm">In particular, the Cirkin has been linked to transports of military material to Libya in May and June 2020.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">21.9.2020</p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">19.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Med Wave Shipping</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Address:</p>
                        <p class="normal">Office 511, 5th Floor, Baraka Building, Dauwar Al-Waha, Jordan;</p>
                        <p class="normal">Adel Al-Hojrat building n<span class="superscript">°</span>3, 1st Floor, opposite Swefieh, Mall-Swefieh Po Box 850880 Amman, 11185 Jordan;</p>
                        <p class="normal">Ground Floor, Orient Queen Homes Building, John Kennedy, Ras Beirut, Lebanon</p>
                        <p class="tbl-norm">Tel.: +962787064121; +96265865550; +96265868550</p>
                        <p class="tbl-norm">E-mail: operation@medwave.co</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Med Wave Shipping is a maritime company which operates a vessel called Bana found to have violated the arms embargo in Libya established in UNSCR 1970 (2011) and transposed in Article 1 of Decision (CFSP) 2015/1333.</p>
                        <p class="tbl-norm">In particular, the Bana has been linked to transports of military material to Libya in January 2020.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">21.9.2020</p>
                     </td>
                  </tr>
               </tbody>
            </table>
         </div>
         <p class="modref">
            <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32016R0044" onclick="window.open(this.href,'_blank'); return false;" title="32016R0044">▼B</a><a class="anchorarrow" id="B-20" href="#B-21"><i class="fa fa-circle" title="LAST" style="text-indent: 5pt;" aria-hidden="true"></i></a>
         </p>
      </div>
"""

extracted_data = extract_table_data(html_content)

# Print the extracted data
for entry in extracted_data:
    print(entry)
    print("---")

{'': '►M27 1. ◄', 'Name': 'ABU SHAARIYA', 'Identifying information': 'Position: Deputy Head, External Security Organisation', 'Reasons': 'Brother-in-law of Muammar Qadhafi. Prominent member of the Qadhafi regime and as such closely associated with the former regime of Muammar Qadhafi.', 'Date of listing': '28.2.2011'}
---
{'': '►M27 2. ◄', 'Name': 'ALSHARGAWI, Bashir Saleh Bashir', 'Identifying information': 'Date of birth: 1946 Place of birth: Traghen', 'Reasons': 'Head of Cabinet of Muammar Qadhafi. Closely associated with the former regime of Muammar Qadhafi.', 'Date of listing': '28.2.2011'}
---
{'': '►M27 4. ◄', 'Name': 'FARKASH, Mohammed Boucharaya', 'Identifying information': 'Date of birth: 1 July 1949 Place of birth: Al-Bayda', 'Reasons': 'Former director of intelligence in External Security Office. Closely associated with the former regime of Muammar Qadhafi.', 'Date of listing': '28.2.2011'}
---
{'': '►M27 5. ◄', 'Name': 'EL-KASSIM ZOUAI, Mohamed Abou', 'Identifying informat

In [ ]:
import re
from bs4 import BeautifulSoup

def clean_text(text):
    return re.sub(r'\s+', ' ', text).strip()

def add_newlines(text):
    text = re.sub(r'([.!?])\s+', r'\1\n\n', text)
    text = re.sub(r'\n{3,}', '\n\n', text)
    return text

def extract_content(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    all_data = []

    # Extract table data
    tables = soup.find_all('table')
    for table in tables:
        headers = [clean_text(th.text) for th in table.find_all('th')]
        if not headers:
            headers = [clean_text(td.text) for td in table.find('tr').find_all('td')]

        rows = table.find_all('tr')[1:]  # Skip the header row
        for row in rows:
            cells = row.find_all('td')
            if len(cells) == len(headers):
                entry = {}
                for header, cell in zip(headers, cells):
                    entry[header] = clean_text(cell.text)
                all_data.append({"type": "table_row", "content": entry})
            all_data.append({"type": "empty_line", "content": ""})

    # Extract paragraph data
    paragraphs = soup.find_all(['p', 'div'])
    for para in paragraphs:
        if para.name == 'p' or (para.name == 'div' and 'class' in para.attrs and 'list' in para['class']):
            text = clean_text(para.text)
            if text:
                all_data.append({"type": "paragraph", "content": add_newlines(text)})

    return all_data

def format_output(extracted_data):
    formatted_output = ""
    for item in extracted_data:
        if item['type'] == 'table_row':
            formatted_output += "Table Row:\n"
            for key, value in item['content'].items():
                formatted_output += f"{key}: {value}\n"
            formatted_output += "\n"
        elif item['type'] == 'paragraph':
            formatted_output += "Paragraph:\n"
            formatted_output += item['content'] + "\n\n"
        elif item['type'] == 'empty_line':
            formatted_output += "\n"
    return formatted_output

# Example usage
html_content = """
<div id="anx_III">
         <p>
            <br>
         </p>
         <hr class="separator-annex">
         <p>
            <br>
         </p>
         <p class="title-annex-1" id="id-9b29d3e7-f299-4373-a9b6-7b45e83fd694">ANNEX III</p>
         <p class="title-gr-seq-level-1" id="id-8cdc45e9-a37f-4a1d-82b2-040fc32dd504">
            <span class="boldface">LIST OF NATURAL AND LEGAL PERSONS, ENTITIES OR BODIES REFERRED TO IN ARTICLE 6(2)</span>
         </p>
         <p class="title-gr-seq-level-2" id="id-079c13d1-4976-4af5-b20a-b6a5d62a6193">A.&nbsp;&nbsp;&nbsp;<span class="boldface">Persons</span>
         </p>
         <p>
            <br>
         </p>
         <div class="centered">
            <p>
               <br>
            </p>
            <table width="960" border="1" cellpadding="2" cellspacing="0" class="borderOj">
               <colgroup><col width="0%">
               <col width="11%">
               <col width="25%">
               <col width="63%">
               <col width="1%">
               </colgroup><tbody>
                  <tr>
                     <td style="font-weight: bold" valign="top">
                        <p class="normal">&nbsp;</p>
                        <div>
                     </div></td>
                     <td style="font-weight: bold" valign="top">
                        <p class="tbl-norm">Name</p>
                     </td>
                     <td style="font-weight: bold" valign="top">
                        <p class="tbl-norm">Identifying information</p>
                     </td>
                     <td style="font-weight: bold" valign="top">
                        <p class="tbl-norm">Reasons</p>
                     </td>
                     <td style="font-weight: bold" valign="top">
                        <p class="tbl-norm">Date of listing</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32019R1292" onclick="window.open(this.href,'_blank'); return false;" title="32019R1292: DELETED">▼M27</a><a class="anchorarrow" id="M27-1" href="#M27-2"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>&nbsp;—————</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32016R0044" onclick="window.open(this.href,'_blank'); return false;" title="32016R0044">▼B</a><a class="anchorarrow" id="B-13" href="#B-14"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>
                        </p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">
                           <span>
                              <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32019R1292" onclick="window.open(this.href,'_blank'); return false;" title="32019R1292: REPLACED">
                                 <span class="boldface">►M27</span>
                              </a><a class="anchorarrow" id="M27-2" href="#M27-3"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>
                           </span>&nbsp;1.<span class="boldface">&nbsp;◄ </span>
                        </p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">ABU SHAARIYA</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Position: Deputy Head, External Security Organisation</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Brother-in-law of Muammar Qadhafi.</p>
                        <p class="tbl-norm">Prominent member of the Qadhafi regime and as such closely associated with the former regime of Muammar Qadhafi.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">28.2.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32018R1073" onclick="window.open(this.href,'_blank'); return false;" title="32018R1073: DELETED">▼M21</a><a class="anchorarrow" id="M21-1" href="#M21-2"><i class="fa fa-circle" title="LAST" style="text-indent: 5pt;" aria-hidden="true"></i></a>&nbsp;—————</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32016R0044" onclick="window.open(this.href,'_blank'); return false;" title="32016R0044">▼B</a><a class="anchorarrow" id="B-14" href="#B-15"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>
                        </p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">
                           <span>
                              <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32019R1292" onclick="window.open(this.href,'_blank'); return false;" title="32019R1292: REPLACED">
                                 <span class="boldface">►M27</span>
                              </a><a class="anchorarrow" id="M27-3" href="#M27-4"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>
                           </span>&nbsp;2.<span class="boldface">&nbsp;◄ </span>
                        </p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">ALSHARGAWI, Bashir Saleh Bashir</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Date of birth: 1946</p>
                        <p class="tbl-norm">Place of birth: Traghen</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Head of Cabinet of Muammar Qadhafi. Closely associated with the former regime of Muammar Qadhafi.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">28.2.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32021R1241" onclick="window.open(this.href,'_blank'); return false;" title="32021R1241: DELETED">▼M36</a><a class="anchorarrow" id="M36-1" href="#M36-2"><i class="fa fa-circle" title="LAST" style="text-indent: 5pt;" aria-hidden="true"></i></a>&nbsp;—————</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32016R0044" onclick="window.open(this.href,'_blank'); return false;" title="32016R0044">▼B</a><a class="anchorarrow" id="B-15" href="#B-16"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>
                        </p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">
                           <span>
                              <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32019R1292" onclick="window.open(this.href,'_blank'); return false;" title="32019R1292: REPLACED">
                                 <span class="boldface">►M27</span>
                              </a><a class="anchorarrow" id="M27-4" href="#M27-5"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>
                           </span>&nbsp;4.<span class="boldface">&nbsp;◄ </span>
                        </p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">FARKASH, Mohammed Boucharaya</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Date of birth: 1 July 1949</p>
                        <p class="tbl-norm">Place of birth: Al-Bayda</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Former director of intelligence in External Security Office.</p>
                        <p class="tbl-norm">Closely associated with the former regime of Muammar Qadhafi.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">28.2.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">
                           <span>
                              <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32019R1292" onclick="window.open(this.href,'_blank'); return false;" title="32019R1292: REPLACED">
                                 <span class="boldface">►M27</span>
                              </a><a class="anchorarrow" id="M27-5" href="#M27-6"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>
                           </span>&nbsp;5.<span class="boldface">&nbsp;◄ </span>
                        </p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">EL-KASSIM ZOUAI, Mohamed Abou</p>
                     </td>
                     <td valign="top">
                        <p class="normal">&nbsp;</p>
                        <div>
                     </div></td>
                     <td valign="top">
                        <p class="tbl-norm">Former Secretary General of the General People's Congress.</p>
                        <p class="tbl-norm">Closely associated with the former regime of Muammar Qadhafi.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">21.3.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32021R1932" onclick="window.open(this.href,'_blank'); return false;" title="32021R1932: DELETED">▼M38</a><a class="anchorarrow" id="M38-1" href="#M38-2"><i class="fa fa-circle" title="LAST" style="text-indent: 5pt;" aria-hidden="true"></i></a>&nbsp;—————</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32016R0044" onclick="window.open(this.href,'_blank'); return false;" title="32016R0044">▼B</a><a class="anchorarrow" id="B-16" href="#B-17"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>
                        </p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">
                           <span>
                              <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32019R1292" onclick="window.open(this.href,'_blank'); return false;" title="32019R1292: REPLACED">
                                 <span class="boldface">►M27</span>
                              </a><a class="anchorarrow" id="M27-6" href="#M27-7"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>
                           </span>&nbsp;7.<span class="boldface">&nbsp;◄ </span>
                        </p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">HIJAZI, Mohamad Mahmoud</p>
                     </td>
                     <td valign="top">
                        <p class="normal">&nbsp;</p>
                        <div>
                     </div></td>
                     <td valign="top">
                        <p class="tbl-norm">Minister for Health and Environment in Colonel Qadhafi's Government.</p>
                        <p class="tbl-norm">Closely associated with the former regime of Muammar Qadhafi.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">21.3.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32023R2501" onclick="window.open(this.href,'_blank'); return false;" title="32023R2501: REPLACED">▼M47</a><a class="anchorarrow" id="M47-1" href="#M47-2"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>
                        </p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">8.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">HOUEJ, Mohamad Ali</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Minister of Economy and Trade in the Government of National Unity of Libya</p>
                        <p class="tbl-norm">Date of birth: 1949</p>
                        <p class="tbl-norm">Place of birth: Al-Azizia (near Tripoli)</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Minister for Industry, Economy and Trade in Colonel Qadhafi’s Government.</p>
                        <p class="tbl-norm">Closely associated with the former regime of Muammar Qadhafi.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">21.3.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32021R0667" onclick="window.open(this.href,'_blank'); return false;" title="32021R0667: DELETED">▼M34</a><a class="anchorarrow" id="M34-1" href="#M34-2"><i class="fa fa-circle" title="LAST" style="text-indent: 5pt;" aria-hidden="true"></i></a>&nbsp;—————</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32016R0044" onclick="window.open(this.href,'_blank'); return false;" title="32016R0044">▼B</a><a class="anchorarrow" id="B-17" href="#B-18"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>
                        </p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">
                           <span>
                              <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32019R1292" onclick="window.open(this.href,'_blank'); return false;" title="32019R1292: REPLACED">
                                 <span class="boldface">►M27</span>
                              </a><a class="anchorarrow" id="M27-7" href="#M27-8"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>
                           </span>&nbsp;10.<span class="boldface">&nbsp;◄ </span>
                        </p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">AL-CHARIF, Ibrahim Zarroug</p>
                     </td>
                     <td valign="top">
                        <p class="normal">&nbsp;</p>
                        <div>
                     </div></td>
                     <td valign="top">
                        <p class="tbl-norm">Minister for Social Affairs in Colonel Qadhafi's Government.</p>
                        <p class="tbl-norm">Closely associated with the former regime of Muammar Qadhafi.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">21.3.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">
                           <span>
                              <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32019R1292" onclick="window.open(this.href,'_blank'); return false;" title="32019R1292: REPLACED">
                                 <span class="boldface">►M27</span>
                              </a><a class="anchorarrow" id="M27-8" href="#M27-9"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>
                           </span>&nbsp;11.<span class="boldface">&nbsp;◄ </span>
                        </p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">FAKHIRI, Abdelkebir Mohamad</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Date of birth: 4 May 1963</p>
                        <p class="tbl-norm">Passport number: B/014965 (expired end 2013)</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Minister for Education, Higher Education and Research in Colonel Qadhafi's Government. Closely associated with the former regime of Muammar Qadhafi.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">21.3.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32023R2501" onclick="window.open(this.href,'_blank'); return false;" title="32023R2501: DELETED">▼M47</a><a class="anchorarrow" id="M47-2" href="#M47-3"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>&nbsp;—————</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32016R1687" onclick="window.open(this.href,'_blank'); return false;" title="32016R1687: DELETED">▼M5</a><a class="anchorarrow" id="M5-1" href="#M5-2"><i class="fa fa-circle" title="LAST" style="text-indent: 5pt;" aria-hidden="true"></i></a>&nbsp;—————</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32019R1292" onclick="window.open(this.href,'_blank'); return false;" title="32019R1292: DELETED">▼M27</a><a class="anchorarrow" id="M27-9" href="#M27-10"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>&nbsp;—————</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32016R0044" onclick="window.open(this.href,'_blank'); return false;" title="32016R0044">▼B</a><a class="anchorarrow" id="B-18" href="#B-19"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>
                        </p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">
                           <span>
                              <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32019R1292" onclick="window.open(this.href,'_blank'); return false;" title="32019R1292: REPLACED">
                                 <span class="boldface">►M27</span>
                              </a><a class="anchorarrow" id="M27-10" href="#M27-11"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>
                           </span>&nbsp;13.<span class="boldface">&nbsp;◄ </span>
                        </p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">DIBRI, Abdulqader Yusef</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Position: Head of Muammar Qadhafi's personal security</p>
                        <p class="tbl-norm">Date of Birth: 1946</p>
                        <p class="tbl-norm">Place of Birth: Houn, Libya</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Responsibility for regime security. History of directing violence against dissidents.</p>
                        <p class="tbl-norm">Closely associated with the former regime of Muammar Qadhafi.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">28.2.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32023R2501" onclick="window.open(this.href,'_blank'); return false;" title="32023R2501: DELETED">▼M47</a><a class="anchorarrow" id="M47-3" href="#M47-4"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>&nbsp;—————</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32022R1308" onclick="window.open(this.href,'_blank'); return false;" title="32022R1308: REPLACED">▼M42</a><a class="anchorarrow" id="M42-1" href="#M42-2"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>
                        </p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">15.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">AL QADHAFI, Quren Salih Quren</p>
                        <p class="tbl-norm">a.k.a. Akrin Akrin Saleh, Al Qadhafi Qurayn Salih Qurayn, Al&nbsp;Qadhafi Qu’ren Salih Qu’ren, Salah Egreen</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Gender: male</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Former Libyan Ambassador to Chad. Has left Chad for Sabha. Involved directly in recruiting and coordinating mercenaries for the regime of the late Muammar Qadhafi.</p>
                        <p class="tbl-norm">Closely associated with the former regime of Muammar Qadhafi.</p>
                        <p class="tbl-norm">Member of the Popular Front for the Liberation of Libya (PFLL), a militia and political party loyal to the late Muammar Qadhafi. Involved in undermining the successful completion of Libya’s political transition by opposing the UN and undermining the UN-facilitated political process, including the Libyan Political Dialogue Forum (LPDF), thereby posing a continued risk to the peace, stability and security of Libya.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">12.4.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32016R0044" onclick="window.open(this.href,'_blank'); return false;" title="32016R0044">▼B</a><a class="anchorarrow" id="B-19" href="#B-20"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>
                        </p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">
                           <span>
                              <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32019R1292" onclick="window.open(this.href,'_blank'); return false;" title="32019R1292: REPLACED">
                                 <span class="boldface">►M27</span>
                              </a><a class="anchorarrow" id="M27-11" href="#M27-12"><i class="fa fa-circle" title="LAST" style="text-indent: 5pt;" aria-hidden="true"></i></a>
                           </span>&nbsp;16.<span class="boldface">&nbsp;◄ </span>
                        </p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">AL KUNI, Colonel Amid Husain</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Believed status/location: South Libya</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Former Governor of Ghat (South Libya). Directly involved in recruiting mercenaries.</p>
                        <p class="tbl-norm">Closely associated with the former regime of Muammar Qadhafi.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">12.4.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32020R1380" onclick="window.open(this.href,'_blank'); return false;" title="32020R1380: DELETED">▼M31</a><a class="anchorarrow" id="M31-1" href="#M31-2"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>&nbsp;—————</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32021R0538" onclick="window.open(this.href,'_blank'); return false;" title="32021R0538: DELETED">▼M33</a><a class="anchorarrow" id="M33-1" href="#M33-2"><i class="fa fa-circle" title="LAST" style="text-indent: 5pt;" aria-hidden="true"></i></a>&nbsp;—————</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32020R1380" onclick="window.open(this.href,'_blank'); return false;" title="32020R1380: DELETED">▼M31</a><a class="anchorarrow" id="M31-2" href="#M31-3"><i class="fa fa-circle" title="LAST" style="text-indent: 5pt;" aria-hidden="true"></i></a>&nbsp;—————</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32022R1308" onclick="window.open(this.href,'_blank'); return false;" title="32022R1308: DELETED">▼M42</a><a class="anchorarrow" id="M42-2" href="#M42-3"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>&nbsp;—————</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32020R1309" onclick="window.open(this.href,'_blank'); return false;" title="32020R1309: INSERTED">▼M30</a><a class="anchorarrow" id="M30-1" href="#M30-2"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>
                        </p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">21.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">DIAB, Moussa</p>
                        <p class="tbl-norm">a.k.a. DIAB, Mousa</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Gender: male</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Moussa Diab is responsible for and has directly engaged in serious human rights abuses including human trafficking and the kidnapping, raping and killing of migrants and refugees.</p>
                        <p class="tbl-norm">He held migrants and refugees in captivity in an illegal detention camp near Bani Walid, where they were treated in an inhuman and degrading manner. Several migrants and refugees were killed when they tried to escape from the detention camp.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">21.9.2020</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32022R1308" onclick="window.open(this.href,'_blank'); return false;" title="32022R1308: REPLACED">▼M42</a><a class="anchorarrow" id="M42-3" href="#M42-4"><i class="fa fa-circle" title="LAST" style="text-indent: 5pt;" aria-hidden="true"></i></a>
                        </p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">22.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Yevgeniy Viktorovich PRIGOZHIN</p>
                        <p class="tbl-norm">(Евгений Викторович Пригожин)</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Date of birth: 1&nbsp;June 1961</p>
                        <p class="tbl-norm">Place of birth: Leningrad, former USSR (now St.&nbsp;Petersburg, Russian Federation)</p>
                        <p class="tbl-norm">Nationality: Russian</p>
                        <p class="tbl-norm">Gender: male</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Yevgeniy Viktorovich Prigozhin is a Russian businessman with close links, including financial, to the Wagner Group, a Russia-based unincorporated private military entity.</p>
                        <p class="tbl-norm">In this way, Prigozhin is engaged in and provides support for the Wagner Group’s activities in Libya, which threaten the country’s peace, stability and security.</p>
                        <p class="tbl-norm">In particular, the Wagner Group is involved in multiple and repeated breaches of the arms embargo in Libya established in UNSCR&nbsp;1970 (2011) and transposed in Article&nbsp;1 of Decision (CFSP)&nbsp;2015/1333, including the delivery of arms and the deployment of mercenaries into Libya in support of the Libya National Army. The Wagner Group has participated in multiple military operations against the UN-endorsed Government of National Accord and has contributed to damaging the stability of Libya and undermining a peaceful process.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">15.10.2020</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32021R2192" onclick="window.open(this.href,'_blank'); return false;" title="32021R2192: INSERTED">▼M39</a><a class="anchorarrow" id="M39-1" href="#M39-2"><i class="fa fa-circle" title="LAST" style="text-indent: 5pt;" aria-hidden="true"></i></a>
                        </p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">23.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">KUZNETSOV, Aleksandr (Alexander) Sergeevich</p>
                        <p class="tbl-norm">(Александр Сергеевич КУЗНЕЦОВ)</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Position (s): Commander of&nbsp;1st Attack and Reconnaissance Company of the Wagner Group</p>
                        <p class="tbl-norm">Callsign: Ratibor</p>
                        <p class="tbl-norm">Date of birth: 8&nbsp;October&nbsp;1977</p>
                        <p class="tbl-norm">Place of birth: Nikolskoye, former USSR (now Russian Federation)</p>
                        <p class="tbl-norm">Nationality: Russian</p>
                        <p class="tbl-norm">Wagner Group ID: M-0271</p>
                        <p class="tbl-norm">Gender: male</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Aleksandr Sergeevich Kuznetsov belongs to the command structure of the Wagner Group, a Russia-based unincorporated private military entity. He took command of the&nbsp;1st Attack and Reconnaissance Company of the Wagner Group in&nbsp;2014. He was injured in&nbsp;September&nbsp;2019 in Libya while fighting alongside the Libyan National Liberation Army (NLA) as commander of the Wagner Group’s military operatives. Through his position and actions, Kuznetsov is responsible for the Wagner Group’s activities that threaten the peace, stability and security of Libya.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">13.12.2021</p>
                     </td>
                  </tr>
               </tbody>
            </table>
         </div>
         <p class="modref">
            <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32016R0044R%2801%29" onclick="window.open(this.href,'_blank'); return false;" title="32016R0044R(01): REPLACED">▼C5</a><a class="anchorarrow" id="C5-1" href="#C5-2"><i class="fa fa-arrow-down" title="NEXT" style="text-indent: 2pt;" aria-hidden="true"></i></a>
         </p>
         <p class="title-gr-seq-level-2" id="id-78988296-a324-4b13-88cc-02a1c6d99b30">B.&nbsp;&nbsp;&nbsp;<span class="boldface">Entities</span>
         </p>
         <p>
            <br>
         </p>
         <div class="centered">
            <p>
               <br>
            </p>
            <table width="960" border="1" cellpadding="2" cellspacing="0" class="borderOj">
               <colgroup><col width="0%">
               <col width="5%">
               <col width="64%">
               <col width="29%">
               <col width="1%">
               </colgroup><tbody>
                  <tr>
                     <td style="font-weight: bold" valign="top">
                        <p class="normal">&nbsp;</p>
                        <div>
                     </div></td>
                     <td style="font-weight: bold" valign="top">
                        <p class="tbl-norm">Name</p>
                     </td>
                     <td style="font-weight: bold" valign="top">
                        <p class="tbl-norm">Identifying information</p>
                     </td>
                     <td style="font-weight: bold" valign="top">
                        <p class="tbl-norm">Reasons</p>
                     </td>
                     <td style="font-weight: bold" valign="top">
                        <p class="tbl-norm">Date of listing</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32022R2525" onclick="window.open(this.href,'_blank'); return false;" title="32022R2525: DELETED">▼M44</a><a class="anchorarrow" id="M44-1" href="#M44-2"><i class="fa fa-circle" title="LAST" style="text-indent: 5pt;" aria-hidden="true"></i></a>&nbsp;—————</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32016R0044R%2801%29" onclick="window.open(this.href,'_blank'); return false;" title="32016R0044R(01): REPLACED">▼C5</a><a class="anchorarrow" id="C5-2" href="#C5-3"><i class="fa fa-circle" title="LAST" style="text-indent: 5pt;" aria-hidden="true"></i></a>
                        </p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">2.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Gaddafi International Charity and Development Foundation</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Contact details of administration: Hay Alandalus — Jian St. — Tripoli — PoBox: 1101 — LIBYA Telephone: (+ 218) 214778301 — Fax (+ 218) 214778766; email: info@gicdf.org</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Closely associated with the former regime of Muammar Qadhafi.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">21.3.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">3.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Waatassimou Foundation</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Based in Tripoli.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Closely associated with the former regime of Muammar Qadhafi.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">21.3.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">4.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Libyan Jamahirya Broadcasting Corporation</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Contact details: tel. 00 218 21 444 59 26; 00 21 444 59 00; fax 00 218 21 340 21 07 http://www.ljbc.net; email: info@ljbc.net</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Closely associated with the former regime of Muammar Qadhafi.</p>
                        <p class="tbl-norm">Involved in public incitement to hatred and violence through participation in disinformation campaigns concerning violence against demonstrators.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">21.3.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">5.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Revolutionary Guard Corps</p>
                     </td>
                     <td valign="top">
                        <p class="normal">&nbsp;</p>
                        <div>
                     </div></td>
                     <td valign="top">
                        <p class="tbl-norm">Closely associated with the former regime of Muammar Qadhafi.</p>
                        <p class="tbl-norm">Involved in violence against demonstrators.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">21.3.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">6.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Libyan Agricultural Bank (a.k.a. Agricultural Bank; a.k.a. Al Masraf Al Zirae Agricultural Bank; a.k.a. Al Masraf Al Zirae; a.k.a. Libyan Agricultural Bank)</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">El Ghayran Area, Ganzor El Sharqya, P.O. Box 1100, Tripoli, Libya; Al Jumhouria Street, East Junzour, Al Gheran, Tripoli, Libya; Email Address agbank@agribankly.org; SWIFT/BIC AGRULYLT (Libya);</p>
                        <p class="tbl-norm">Tel. No. (218)214870586;</p>
                        <p class="tbl-norm">Tel. No. (218) 214870714;</p>
                        <p class="tbl-norm">Tel. No. (218) 214870745;</p>
                        <p class="tbl-norm">Tel. No. (218) 213338366;</p>
                        <p class="tbl-norm">Tel. No. (218) 213331533;</p>
                        <p class="tbl-norm">Tel. No. (218) 213333541;</p>
                        <p class="tbl-norm">Tel. No. (218) 213333544;</p>
                        <p class="tbl-norm">Tel. No. (218) 213333543;</p>
                        <p class="tbl-norm">Tel. No. (218) 213333542;</p>
                        <p class="tbl-norm">Fax No. (218) 214870747;</p>
                        <p class="tbl-norm">Fax No. (218) 214870767;</p>
                        <p class="tbl-norm">Fax No. (218) 214870777;</p>
                        <p class="tbl-norm">Fax No. (218) 213330927;</p>
                        <p class="tbl-norm">Fax No. (218) 213333545</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Libyan subsidiary of the Central Bank of Libya.</p>
                        <p class="tbl-norm">Closely associated with the former regime of Muammar Qadhafi.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">12.4.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">7.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Al-Inma Holding Co. for Services Investments</p>
                     </td>
                     <td valign="top">
                        <p class="normal">&nbsp;</p>
                        <div>
                     </div></td>
                     <td valign="top">
                        <p class="tbl-norm">Libyan subsidiary of the Economic &amp; Social Development Fund.</p>
                        <p class="tbl-norm">Closely associated with the former regime of Muammar Qadhafi.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">12.4.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">8.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Al-Inma Holding Co. For Industrial Investments</p>
                     </td>
                     <td valign="top">
                        <p class="normal">&nbsp;</p>
                        <div>
                     </div></td>
                     <td valign="top">
                        <p class="tbl-norm">Libyan subsidiary of the Economic &amp; Social Development Fund.</p>
                        <p class="tbl-norm">Closely associated with the former regime of Muammar Qadhafi.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">12.4.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">9.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Al-Inma Holding Company for Tourism Investment</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Hasan al-Mashay Street (off al- Zawiyah Street) Tel.&nbsp;No.:&nbsp;(218)&nbsp;213345187 Fax +218.21.334.5188 email:&nbsp;info@ethic.ly</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Libyan subsidiary of the Economic &amp; Social Development Fund.</p>
                        <p class="tbl-norm">Closely associated with the former regime of Muammar Qadhafi.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">12.4.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">10.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Al-Inma Holding Co. for Construction and Real Estate Developments</p>
                     </td>
                     <td valign="top">
                        <p class="normal">&nbsp;</p>
                        <div>
                     </div></td>
                     <td valign="top">
                        <p class="tbl-norm">Libyan subsidiary of the Economic &amp; Social Development Fund.</p>
                        <p class="tbl-norm">Closely associated with the former regime of Muammar Qadhafi.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">12.4.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">11.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">LAP Green Networks (a.k.a. Lap GreenN, LAP Green Holding Company)</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">9th Floor, Ebene Tower, 52, Cybercity, Ebene, Mauritius</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Libyan subsidiary of the Libyan Africa Investment Portfolio.</p>
                        <p class="tbl-norm">Closely associated with the former regime of Muammar Qadhafi.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">12.4.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">12.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Sabtina Ltd</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">530-532 Elder Gate, Elder House, Milton Keynes, UK</p>
                        <p class="tbl-norm">Other info: Reg no 01794877 (UK)</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">UK-incorporated subsidiary of the Libyan Investment Authority.</p>
                        <p class="tbl-norm">Closely associated with the former regime of Muammar Qadhafi.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">12.4.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">13.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Ashton Global Investments Limited</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Woodbourne Hall, PO Box 3162, Road Town, Tortola, British Virgin Islands</p>
                        <p class="tbl-norm">Other Info: Reg no 1510484 (BVI)</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">BVI-incorporated subsidiary of the Libyan Investment Authority.</p>
                        <p class="tbl-norm">Closely associated with the former regime of Muammar Qadhafi.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">12.4.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">14.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Capitana Seas Limited</p>
                     </td>
                     <td valign="top">
                        <p class="normal">&nbsp;</p>
                        <div>
                     </div></td>
                     <td valign="top">
                        <p class="tbl-norm">BVI-incorporated entity owned by Saadi Qadhafi</p>
                        <p class="tbl-norm">Closely associated with the former regime of Muammar Qadhafi.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">12.4.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">15.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Kinloss Property Limited</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Woodbourne Hall, PO Box 3162, Road Town, Tortola, British Virgin Islands</p>
                        <p class="tbl-norm">Other Info: Reg no 1534407 (BVI)</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">BVI-incorporated subsidiary of the Libyan Investment Authority.</p>
                        <p class="tbl-norm">Closely associated with the former regime of Muammar Qadhafi.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">12.4.2011</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32023R2501" onclick="window.open(this.href,'_blank'); return false;" title="32023R2501: DELETED">▼M47</a><a class="anchorarrow" id="M47-4" href="#M47-5"><i class="fa fa-circle" title="LAST" style="text-indent: 5pt;" aria-hidden="true"></i></a>&nbsp;—————</p>
                     </td>
                  </tr>
                  <tr>
                     <td colspan="5">
                        <p class="modref">
                           <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32020R1309" onclick="window.open(this.href,'_blank'); return false;" title="32020R1309: INSERTED">▼M30</a><a class="anchorarrow" id="M30-2" href="#M30-3"><i class="fa fa-circle" title="LAST" style="text-indent: 5pt;" aria-hidden="true"></i></a>
                        </p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">17.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Sigma Airlines</p>
                        <p class="tbl-norm">a.k.a. Sigma Aviation; Air Sigma</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Address:</p>
                        <p class="normal">Markov Str. 11</p>
                        <p class="normal">050013, Almaty, Kazakhstan</p>
                        <p class="tbl-norm">Tel.: +77272922305</p>
                        <p class="tbl-norm">Website: https://airsigma.pro/</p>
                        <p class="tbl-norm">Registered under name: Kenesbayev Umirbek Zharmenovich</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Sigma Airlines is a commercial cargo air company which operates aircrafts found to have violated the arms embargo in Libya established in UNSCR 1970 (2011) and transposed in Article 1 of Decision (CFSP) 2015/1333.</p>
                        <p class="tbl-norm">The UN has identified Sigma Airlines as one of the commercial air cargo providers operating in non-compliance with the UN embargo on the transfer of military material to Libya.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">21.9.2020</p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">18.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Avrasya Shipping</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Address:</p>
                        <p class="tbl-norm">Liman Mh. Gezi Cd. No:22/3 İlkadım, Samsun, Turkey</p>
                        <p class="tbl-norm">Tel.: +90 549 720 1748</p>
                        <p class="tbl-norm">E-mail: info@avrasyashipping.com</p>
                        <p class="tbl-norm">Website: http://www.avrasyashipping.com/iletisim</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Avrasya Shipping is a maritime company which operates a vessel called Cirkin found to have violated the arms embargo in Libya established in UNSCR 1970 (2011) and transposed in Article 1 of Decision (CFSP) 2015/1333.</p>
                        <p class="tbl-norm">In particular, the Cirkin has been linked to transports of military material to Libya in May and June 2020.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">21.9.2020</p>
                     </td>
                  </tr>
                  <tr>
                     <td valign="top">
                        <p class="tbl-norm">19.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Med Wave Shipping</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Address:</p>
                        <p class="normal">Office 511, 5th Floor, Baraka Building, Dauwar Al-Waha, Jordan;</p>
                        <p class="normal">Adel Al-Hojrat building n<span class="superscript">°</span>3, 1st Floor, opposite Swefieh, Mall-Swefieh Po Box 850880 Amman, 11185 Jordan;</p>
                        <p class="normal">Ground Floor, Orient Queen Homes Building, John Kennedy, Ras Beirut, Lebanon</p>
                        <p class="tbl-norm">Tel.: +962787064121; +96265865550; +96265868550</p>
                        <p class="tbl-norm">E-mail: operation@medwave.co</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-norm">Med Wave Shipping is a maritime company which operates a vessel called Bana found to have violated the arms embargo in Libya established in UNSCR 1970 (2011) and transposed in Article 1 of Decision (CFSP) 2015/1333.</p>
                        <p class="tbl-norm">In particular, the Bana has been linked to transports of military material to Libya in January 2020.</p>
                     </td>
                     <td valign="top">
                        <p class="tbl-left">21.9.2020</p>
                     </td>
                  </tr>
               </tbody>
            </table>
         </div>
         <p class="modref">
            <a href="./../../../../legal-content/EN/AUTO/?uri=celex:32016R0044" onclick="window.open(this.href,'_blank'); return false;" title="32016R0044">▼B</a><a class="anchorarrow" id="B-20" href="#B-21"><i class="fa fa-circle" title="LAST" style="text-indent: 5pt;" aria-hidden="true"></i></a>
         </p>
      </div>
"""

extracted_data = extract_content(html_content)
data = format_output(extracted_data)

# Save the data to a text file
with open('extracted_content.txt', 'w', encoding='utf-8') as file:
    file.write(data)

print("Data has been saved to 'extracted_content.txt'")

Data has been saved to 'extracted_content.txt'


In [ ]:
data

"\n\nTable Row:\n: ►M27 1. ◄\nName: ABU SHAARIYA\nIdentifying information: Position: Deputy Head, External Security Organisation\nReasons: Brother-in-law of Muammar Qadhafi. Prominent member of the Qadhafi regime and as such closely associated with the former regime of Muammar Qadhafi.\nDate of listing: 28.2.2011\n\n\n\n\nTable Row:\n: ►M27 2. ◄\nName: ALSHARGAWI, Bashir Saleh Bashir\nIdentifying information: Date of birth: 1946 Place of birth: Traghen\nReasons: Head of Cabinet of Muammar Qadhafi. Closely associated with the former regime of Muammar Qadhafi.\nDate of listing: 28.2.2011\n\n\n\n\nTable Row:\n: ►M27 4. ◄\nName: FARKASH, Mohammed Boucharaya\nIdentifying information: Date of birth: 1 July 1949 Place of birth: Al-Bayda\nReasons: Former director of intelligence in External Security Office. Closely associated with the former regime of Muammar Qadhafi.\nDate of listing: 28.2.2011\n\n\nTable Row:\n: ►M27 5. ◄\nName: EL-KASSIM ZOUAI, Mohamed Abou\nIdentifying information: \nReaso

In [ ]:
from transformers import BertTokenizerFast, BertForTokenClassification
import torch

# Load the fine-tuned model and tokenizer
model_dir = "./ner_model"  # Directory where the fine-tuned model is saved
tokenizer = BertTokenizerFast.from_pretrained(model_dir)
model = BertForTokenClassification.from_pretrained(model_dir)

# Set the model to evaluation mode
model.eval()

# Define the string to perform NER on
x = """Title: na Designation: Leader of a transnational trafficking network DOB: 19 Jan. 1983 POB: Sabratha, Libya Good quality a.k.a.: Mus'ab Abu Qarin Low quality a.k.a.: a) ABU-AL QASSIM OMAR Musab Boukrin b) The Doctor c) Al-Grein Nationality: Libya Passport no: a) 782633, issued on 31 May 2005 b) 540794, issued on 12 Jan. 2008 National identification no: na Address: na Listed on: 7 June 2018 Other information: Listed pursuant to paragraphs 15 and 17 of resolution 1970 (Travel Ban, Asset Freeze)"""

# Tokenize the text in variable x
inputs = tokenizer(x, return_tensors="pt", truncation=True, padding=True, is_split_into_words=False)

# Perform NER using the model
with torch.no_grad():
    outputs = model(**inputs)

# Get predictions (logits) and convert them into label IDs
predictions = torch.argmax(outputs.logits, dim=2)

# List of entity labels (ensure it's the same as used during fine-tuning)
label_list = ["O", "B-PERSON", "I-PERSON", "B-PERSON_ARABIC", "I-PERSON_ARABIC", "B-ALIAS", "I-ALIAS",
              "B-DATE_OF_BIRTH", "I-DATE_OF_BIRTH", "B-PLACE_OF_BIRTH", "I-PLACE_OF_BIRTH",
              "B-COUNTRY", "I-COUNTRY", "B-FUNCTION", "I-FUNCTION", "B-RANK", "I-RANK",
              "B-NATIONALITY", "I-NATIONALITY", "B-GENDER", "I-GENDER", "B-PASSPORT_NUMBER",
              "I-PASSPORT_NUMBER", "B-REASON_FOR_LISTING", "I-REASON_FOR_LISTING", "B-DATE_OF_LISTING",
              "I-DATE_OF_LISTING", "B-UN_LISTING_DATE", "I-UN_LISTING_DATE"]

# Convert token IDs back to tokens
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

# Convert predicted label IDs back to label names
predicted_labels = [label_list[p.item()] for p in predictions[0]]

# Clean up the output by handling special tokens and subword tokens
output = []
current_entity = []  # To group multi-token entities like date of birth
previous_label = "O"
previous_token_is_subword = False

for token, label in zip(tokens, predicted_labels):
    # Skip special tokens like [CLS] and [SEP]
    if token in ["[CLS]", "[SEP]"]:
        continue

    # Join subword tokens with the previous token
    if token.startswith("##"):
        if previous_token_is_subword:
            current_entity[-1] = f"{current_entity[-1]}{token[2:]}"
        else:
            current_entity[-1] = f"{current_entity[-1]}{token[2:]}"
        previous_token_is_subword = True
    else:
        # Only include tokens with a relevant entity label
        if label.startswith("B-"):
            if current_entity:
                output.append(" ".join(current_entity))  # Append completed entity
            current_entity = [f"{token} ['{label}']"]  # Start new entity
            previous_label = label
        elif label.startswith("I-") and previous_label[2:] == label[2:]:
            # Only include 'I-' labels if they follow the correct 'B-' label
            current_entity.append(f"{token} ['{label}']")
        else:
            if current_entity:
                output.append(" ".join(current_entity))  # Append completed entity
            output.append(token)  # Normal token without entity
            current_entity = []
            previous_label = "O"
        previous_token_is_subword = False

# Append the last entity if there is one
if current_entity:
    output.append(" ".join(current_entity))

# Join the tokens into a sentence with spaces
output_text = " ".join(output)
print(output_text)


IndexError: list index out of range

In [ ]:
from transformers import BertTokenizerFast, BertForTokenClassification
import torch

# Load the fine-tuned model and tokenizer
model_dir = "./ner_model"  # Directory where the fine-tuned model is saved
tokenizer = BertTokenizerFast.from_pretrained(model_dir)
model = BertForTokenClassification.from_pretrained(model_dir)

# Set the model to evaluation mode
model.eval()

# Define the string to perform NER on
x = "Joshua is one of the most prominent figures and was born on 12-12-1990."

# Tokenize the text in variable x
inputs = tokenizer(x, return_tensors="pt", truncation=True, padding=True, is_split_into_words=False)

# Perform NER using the model
with torch.no_grad():
    outputs = model(**inputs)

# Get predictions (logits) and convert them into label IDs
predictions = torch.argmax(outputs.logits, dim=2)

# List of entity labels (ensure it's the same as used during fine-tuning)
label_list = ["O", "B-PERSON", "I-PERSON", "B-PERSON_ARABIC", "I-PERSON_ARABIC", "B-ALIAS", "I-ALIAS",
              "B-DATE_OF_BIRTH", "I-DATE_OF_BIRTH", "B-PLACE_OF_BIRTH", "I-PLACE_OF_BIRTH",
              "B-COUNTRY", "I-COUNTRY", "B-FUNCTION", "I-FUNCTION", "B-RANK", "I-RANK",
              "B-NATIONALITY", "I-NATIONALITY", "B-GENDER", "I-GENDER", "B-PASSPORT_NUMBER",
              "I-PASSPORT_NUMBER", "B-REASON_FOR_LISTING", "I-REASON_FOR_LISTING", "B-DATE_OF_LISTING",
              "I-DATE_OF_LISTING", "B-UN_LISTING_DATE", "I-UN_LISTING_DATE"]

# Convert token IDs back to tokens
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

# Convert predicted label IDs back to label names
predicted_labels = [label_list[p.item()] for p in predictions[0]]

# Clean up the output by handling special tokens and subword tokens
output = []
current_entity = []  # To group multi-token entities like date of birth
current_label = None
previous_label = "O"
previous_token_is_subword = False

for token, label in zip(tokens, predicted_labels):
    # Skip special tokens like [CLS] and [SEP]
    if token in ["[CLS]", "[SEP]"]:
        continue

    # Join subword tokens with the previous token
    if token.startswith("##"):
        if current_entity:
            current_entity[-1] = f"{current_entity[-1]}{token[2:]}"
        else:
            current_entity.append(token[2:])
        previous_token_is_subword = True
    else:
        # When encountering a new entity, add the current entity to the output
        if label != "O" and label.startswith("B-"):
            if current_entity:
                # Merge tokens and add to the output
                output.append(f"{''.join(current_entity)} ['{current_label}']")
            current_entity = [token]  # Start a new entity
            current_label = label  # Set the current label
        elif label.startswith("I-") and label[2:] == current_label[2:]:
            # Continue adding to the current entity if the label is the same
            current_entity.append(token)
        else:
            if current_entity:
                # Merge tokens and add to the output
                output.append(f"{''.join(current_entity)} ['{current_label}']")
            output.append(token)  # Non-entity token
            current_entity = []
            current_label = None
        previous_token_is_subword = False

# Append the last entity if it exists
if current_entity:
    output.append(f"{''.join(current_entity)} ['{current_label}']")

# Join the tokens into a sentence with spaces
output_text = " ".join(output)
print(output_text)


TypeError: 'NoneType' object is not subscriptable

In [ ]:
from transformers import BertTokenizerFast, BertForTokenClassification
import torch

# Load the fine-tuned model and tokenizer
model_dir = "./ner_model"  # Directory where the fine-tuned model is saved
tokenizer = BertTokenizerFast.from_pretrained(model_dir)
model = BertForTokenClassification.from_pretrained(model_dir)

# Set the model to evaluation mode
model.eval()

# Define the string to perform NER on
x = """Title: na Designation: a) Director, External Security Organisation. b) Head of external intelligence agency. DOB: 4 Apr. 1944 POB: Alrhaybat Good quality a.k.a.: a) Dorda Abuzed OE b) Abu Zayd Umar Hmeid Dorda Low quality a.k.a.: na Nationality: na Passport no: Libya number FK117RK0, issued on 25 Nov. 2018, issued in Tripoli (Date of expiration: 24 Nov 2026) National identification no: na Address: Libya (Believed status/location: deceased) Listed on: 26 Feb. 2011 (amended on 27 Jun. 2014, 1 Apr. 2016, 25 Feb. 2020, 18 Jul. 2022) Other information: Listed pursuant to paragraph 15 of resolution 1970 (Travel Ban). Listed on 17 March 2011 pursuant to paragraph 17 of resolution 1970 (Asset Freeze)."""

# Tokenize the text in variable x
inputs = tokenizer(x, return_tensors="pt", truncation=True, padding=True, is_split_into_words=False)

# Perform NER using the model
with torch.no_grad():
    outputs = model(**inputs)

# Get predictions (logits) and convert them into label IDs
predictions = torch.argmax(outputs.logits, dim=2)

# List of entity labels (ensure it's the same as used during fine-tuning)
label_list = ["O", "B-PERSON", "I-PERSON", "B-PERSON_ARABIC", "I-PERSON_ARABIC", "B-ALIAS", "I-ALIAS",
              "B-DATE_OF_BIRTH", "I-DATE_OF_BIRTH", "B-PLACE_OF_BIRTH", "I-PLACE_OF_BIRTH",
              "B-COUNTRY", "I-COUNTRY", "B-FUNCTION", "I-FUNCTION", "B-RANK", "I-RANK",
              "B-NATIONALITY", "I-NATIONALITY", "B-GENDER", "I-GENDER", "B-PASSPORT_NUMBER",
              "I-PASSPORT_NUMBER", "B-REASON_FOR_LISTING", "I-REASON_FOR_LISTING", "B-DATE_OF_LISTING",
              "I-DATE_OF_LISTING", "B-UN_LISTING_DATE", "I-UN_LISTING_DATE"]

# Convert token IDs back to tokens
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

# Convert predicted label IDs back to label names
predicted_labels = [label_list[p.item()] if p is not None else 'O' for p in predictions[0]]  # Ensure p is not None

# Clean up the output by handling special tokens and subword tokens
output = []
current_entity = []  # To group multi-token entities like date of birth
current_label = None
previous_label = "O"
previous_token_is_subword = False

for token, label in zip(tokens, predicted_labels):
    # Ensure that both token and label are valid, skip if None
    if token is None or label is None:
        continue

    # Skip special tokens like [CLS] and [SEP]
    if token in ["[CLS]", "[SEP]"]:
        continue

    # Join subword tokens with the previous token
    if token.startswith("##"):
        if current_entity:
            current_entity[-1] = f"{current_entity[-1]}{token[2:]}"
        else:
            current_entity.append(token[2:])
        previous_token_is_subword = True
    else:
        # When encountering a new entity, add the current entity to the output
        if label != "O" and label.startswith("B-"):
            if current_entity:
                # Merge tokens and add to the output
                output.append(f"{''.join(current_entity)} ['{current_label}']")
            current_entity = [token]  # Start a new entity
            current_label = label  # Set the current label
        elif label.startswith("I-") and current_label and label[2:] == current_label[2:]:
            # Continue adding to the current entity if the label is the same
            current_entity.append(token)
        else:
            if current_entity:
                # Merge tokens and add to the output
                output.append(f"{''.join(current_entity)} ['{current_label}']")
            output.append(token)  # Non-entity token
            current_entity = []
            current_label = None
        previous_token_is_subword = False

# Append the last entity if it exists
if current_entity:
    output.append(f"{''.join(current_entity)} ['{current_label}']")

# Join the tokens into a sentence with spaces
output_text = " ".join(output)
print(output_text)


Title : na Design ation ['None'] : a ) Director , External Security Organisation . b ) Head of external intelligence agency . D OB ['None'] : 4Apr.1944 ['B-DATE_OF_BIRTH'] P OB ['None'] : Alrhaybat ['B-PLACE_OF_BIRTH'] Good quality a . k . a . : a ) Do rda ['None'] Abu zed ['None'] O E ['None'] b ) Abu Z ayd ['None'] Um ar ['None'] H meid ['None'] Do rda ['None'] Low quality a . k . a . : na National ity ['None'] : na Pass port ['None'] no : Libya number FK117RK0 ['B-PASSPORT_NUMBER'] , issued on 25Nov.2018 ['B-DATE_OF_LISTING'] , issued in Tripoli ( Date of ex piration ['None'] : 24Nov2026 ['B-DATE_OF_LISTING'] ) National identification no : na Ad dress ['None'] : Libya ( Believe d ['None'] status / location : deceased ) Listed on : 26Feb.2011 ['B-DATE_OF_LISTING'] ( amended on 27 ['B-UN_LISTING_DATE'] Jun . 2014 , 1 ['B-UN_LISTING_DATE'] Apr . 2016 , 25 ['B-UN_LISTING_DATE'] Feb . 2020 , 18 ['B-UN_LISTING_DATE'] Jul . 202 2 ['None'] ) Other information : Listed pu rsuant ['None'] to 

In [ ]:
from transformers import BertTokenizerFast, BertForTokenClassification
import torch

# Load the fine-tuned model and tokenizer
model_dir = "./ner_model"  # Directory where the fine-tuned model is saved
tokenizer = BertTokenizerFast.from_pretrained(model_dir)
model = BertForTokenClassification.from_pretrained(model_dir)

# Set the model to evaluation mode
model.eval()

# Define the string to perform NER on
x = """6.    Name: 1: ABU 2: ZAYD 3: UMAR 4: DORDA

Title: na Designation: a) Director, External Security Organisation. b) Head of external intelligence agency. DOB: 4 Apr. 1944 POB: Alrhaybat Good quality a.k.a.: a) Dorda Abuzed OE b) Abu Zayd Umar Hmeid Dorda Low quality a.k.a.: na Nationality: na Passport no: Libya number FK117RK0, issued on 25 Nov. 2018, issued in Tripoli (Date of expiration: 24 Nov 2026) National identification no: na Address: Libya (Believed status/location: deceased) Listed on: 26 Feb. 2011 (amended on 27 Jun. 2014, 1 Apr. 2016, 25 Feb. 2020, 18 Jul. 2022) Other information: Listed pursuant to paragraph 15 of resolution 1970 (Travel Ban). Listed on 17 March 2011 pursuant to paragraph 17 of resolution 1970 (Asset Freeze)."""
# Tokenize the text in variable x
inputs = tokenizer(x, return_tensors="pt", truncation=True, padding=True, is_split_into_words=False)

# Perform NER using the model
with torch.no_grad():
    outputs = model(**inputs)

# Get predictions (logits) and convert them into label IDs
predictions = torch.argmax(outputs.logits, dim=2)

# List of entity labels (ensure it's the same as used during fine-tuning)
label_list = ["O", "B-PERSON", "I-PERSON", "B-PERSON_ARABIC", "I-PERSON_ARABIC", "B-ALIAS", "I-ALIAS",
              "B-DATE_OF_BIRTH", "I-DATE_OF_BIRTH", "B-PLACE_OF_BIRTH", "I-PLACE_OF_BIRTH",
              "B-COUNTRY", "I-COUNTRY", "B-FUNCTION", "I-FUNCTION", "B-RANK", "I-RANK",
              "B-NATIONALITY", "I-NATIONALITY", "B-GENDER", "I-GENDER", "B-PASSPORT_NUMBER",
              "I-PASSPORT_NUMBER", "B-REASON_FOR_LISTING", "I-REASON_FOR_LISTING", "B-DATE_OF_LISTING",
              "I-DATE_OF_LISTING", "B-UN_LISTING_DATE", "I-UN_LISTING_DATE"]

# Convert token IDs back to tokens
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

# Fix invalid label lookups by using the index safely
predicted_labels = []
for p in predictions[0]:
    try:
        predicted_labels.append(label_list[p.item()])
    except IndexError:
        predicted_labels.append("O")  # Default to 'O' for out-of-bounds predictions

# Clean up the output by handling special tokens and subword tokens
output = []
current_entity = []  # To group multi-token entities like date of birth
current_label = None
previous_label = "O"
previous_token_is_subword = False

for token, label in zip(tokens, predicted_labels):
    # Ensure that both token and label are valid, skip if None
    if token is None or label is None:
        continue

    # Skip special tokens like [CLS] and [SEP]
    if token in ["[CLS]", "[SEP]"]:
        continue

    # Join subword tokens with the previous token
    if token.startswith("##"):
        if current_entity:
            current_entity[-1] = f"{current_entity[-1]}{token[2:]}"
        else:
            current_entity.append(token[2:])
        previous_token_is_subword = True
    else:
        # When encountering a new entity, add the current entity to the output
        if label != "O" and label.startswith("B-"):
            if current_entity:
                # Merge tokens and add to the output
                output.append(f"{''.join(current_entity)} ['{current_label}']")
            current_entity = [token]  # Start a new entity
            current_label = label  # Set the current label
        elif label.startswith("I-") and current_label and label[2:] == current_label[2:]:
            # Continue adding to the current entity if the label is the same
            current_entity.append(token)
        else:
            if current_entity:
                # Merge tokens and add to the output
                output.append(f"{''.join(current_entity)} ['{current_label}']")
            output.append(token)  # Non-entity token
            current_entity = []
            current_label = None
        previous_token_is_subword = False

# Append the last entity if it exists
if current_entity:
    output.append(f"{''.join(current_entity)} ['{current_label}']")

# Join the tokens into a sentence with spaces
output_text = " ".join(output)
print(output_text)


6 ['B-PERSON'] . Name : 1 : AB U ['None'] 2 : Z AYD ['None'] 3 : U MAR ['None'] 4 : D ORDA ['None'] Title : na Design ation ['None'] : a ) Director , External Security Organisation . b ) Head of external intelligence agency . D OB ['None'] : 4Apr.1944 ['B-DATE_OF_BIRTH'] P OB ['None'] : Al rhaybat ['None'] Good quality a . k . a . : a ) Do rda ['None'] Abu zed ['None'] O E ['None'] b ) Abu Z ayd ['None'] Um ar ['None'] H meid ['None'] Do rda ['None'] Low quality a . k . a . : na National ity ['None'] : na Pass port ['None'] no : Libya number FK117RK0 ['B-PASSPORT_NUMBER'] , issued on 25Nov.2018 ['B-DATE_OF_LISTING'] , issued in Tripoli ['B-COUNTRY'] ( Date of ex piration ['None'] : 24Nov2026 ['B-DATE_OF_LISTING'] ) National identification no : na Ad dress ['None'] : Libya ( Believe d ['None'] status / location : deceased ) Listed on : 26Feb.2011 ['B-DATE_OF_LISTING'] ( amended on 27 ['B-UN_LISTING_DATE'] Jun . 2014 , 1 ['B-UN_LISTING_DATE'] Apr . 2016 , 25 ['B-UN_LISTING_DATE'] Feb . 2

In [ ]:
from transformers import BertTokenizerFast, BertForTokenClassification
import torch

# Load the fine-tuned model and tokenizer
model_dir = "./ner_model"  # Directory where the fine-tuned model is saved
tokenizer = BertTokenizerFast.from_pretrained(model_dir)
model = BertForTokenClassification.from_pretrained(model_dir)

# Set the model to evaluation mode
model.eval()

# Define the string to perform NER on
x = """6.    Name: 1: ABU 2: ZAYD 3: UMAR 4: DORDA

Title: na Designation: a) Director, External Security Organisation. b) Head of external intelligence agency. DOB: 4 Apr. 1944 POB: Alrhaybat Good quality a.k.a.: a) Dorda Abuzed OE b) Abu Zayd Umar Hmeid Dorda Low quality a.k.a.: na Nationality: na Passport no: Libya number FK117RK0, issued on 25 Nov. 2018, issued in Tripoli (Date of expiration: 24 Nov 2026) National identification no: na Address: Libya (Believed status/location: deceased) Listed on: 26 Feb. 2011 (amended on 27 Jun. 2014, 1 Apr. 2016, 25 Feb. 2020, 18 Jul. 2022) Other information: Listed pursuant to paragraph 15"""
# Tokenize the text in variable x
inputs = tokenizer(x, return_tensors="pt", truncation=True, padding=True, is_split_into_words=False)

# Perform NER using the model
with torch.no_grad():
    outputs = model(**inputs)

# Get predictions (logits) and convert them into label IDs
predictions = torch.argmax(outputs.logits, dim=2)

# List of entity labels (ensure it's the same as used during fine-tuning)
label_list = ["O", "B-PERSON", "I-PERSON", "B-PERSON_ARABIC", "I-PERSON_ARABIC", "B-ALIAS", "I-ALIAS",
              "B-DATE_OF_BIRTH", "I-DATE_OF_BIRTH", "B-PLACE_OF_BIRTH", "I-PLACE_OF_BIRTH",
              "B-COUNTRY", "I-COUNTRY", "B-FUNCTION", "I-FUNCTION", "B-RANK", "I-RANK",
              "B-NATIONALITY", "I-NATIONALITY", "B-GENDER", "I-GENDER", "B-PASSPORT_NUMBER",
              "I-PASSPORT_NUMBER", "B-REASON_FOR_LISTING", "I-REASON_FOR_LISTING", "B-DATE_OF_LISTING",
              "I-DATE_OF_LISTING", "B-UN_LISTING_DATE", "I-UN_LISTING_DATE"]

# Normalize certain labels (e.g., remove B- and I- for gender and other entities)
def normalize_label(label):
    if label in ["B-GENDER", "I-GENDER"]:
        return "GENDER"
    elif label.startswith("B-") or label.startswith("I-"):
        return label[2:]  # Remove the B- and I- prefix for other labels
    return label  # Return 'O' or other labels unchanged

# Convert token IDs back to tokens
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

# Fix invalid label lookups by using the index safely and normalize labels
predicted_labels = []
for p in predictions[0]:
    try:
        predicted_labels.append(normalize_label(label_list[p.item()]))
    except IndexError:
        predicted_labels.append("O")  # Default to 'O' for out-of-bounds predictions

# Clean up the output by handling special tokens and subword tokens
output = []
current_entity = []  # To group multi-token entities like date of birth
current_label = None
previous_label = "O"
previous_token_is_subword = False

for token, label in zip(tokens, predicted_labels):
    # Ensure that both token and label are valid, skip if None
    if token is None or label is None:
        continue

    # Skip special tokens like [CLS] and [SEP]
    if token in ["[CLS]", "[SEP]"]:
        continue

    # Join subword tokens with the previous token
    if token.startswith("##"):
        if current_entity:
            current_entity[-1] = f"{current_entity[-1]}{token[2:]}"
        else:
            current_entity.append(token[2:])
        previous_token_is_subword = True
    else:
        # When encountering a new entity, add the current entity to the output
        if label != "O" and current_label != label:
            if current_entity:
                # Merge tokens and add to the output
                output.append(f"{''.join(current_entity)} ['{current_label}']")
            current_entity = [token]  # Start a new entity
            current_label = label  # Set the current label
        elif label == current_label:
            # Continue adding to the current entity if the label is the same
            current_entity.append(token)
        else:
            if current_entity:
                # Merge tokens and add to the output
                output.append(f"{''.join(current_entity)} ['{current_label}']")
            output.append(token)  # Non-entity token
            current_entity = []
            current_label = None
        previous_token_is_subword = False

# Append the last entity if it exists
if current_entity:
    output.append(f"{''.join(current_entity)} ['{current_label}']")

# Join the tokens into a sentence with spaces
output_text = " ".join(output)
print(output_text)


6 ['PERSON'] . Name : 1 : AB U ['None'] 2 : Z AYD ['None'] 3 : U MAR ['None'] 4 : D ORDA ['None'] Title : na Design ation ['None'] : a)Director,ExternalSecurityOrganisation ['FUNCTION'] . b )Headofexternalintelligenceagency ['FUNCTION'] . D OB ['None'] : 4Apr.1944 ['DATE_OF_BIRTH'] P OB ['None'] : Alrhaybat ['NATIONALITY'] Good ['REASON_FOR_LISTING'] quality a . k . a . : a ) Dorda ['REASON_FOR_LISTING'] Abu zed ['None'] O E ['None'] b ) Abu ZaydUmar ['REASON_FOR_LISTING'] Hmeid ['PERSON_ARABIC'] Do rda ['None'] Low quality a . k . a . : na National ity ['None'] : na Pass port ['None'] no : Libya number FK117RK0 ['PASSPORT_NUMBER'] , issued on 25Nov.2018 ['DATE_OF_LISTING'] , issued in Tripoli ['COUNTRY'] ( Date of ex piration ['None'] : 24Nov2026 ['DATE_OF_LISTING'] ) National identification no : na Ad dress ['None'] : Libya ['NATIONALITY'] ( Believe d ['None'] status / location : deceased ) Listed on : 26Feb.2011 ['DATE_OF_LISTING'] ( amended on 27 ['UN_LISTING_DATE'] Jun.2014 ['DATE

In [ ]:
from transformers import BertTokenizerFast, BertForTokenClassification
import torch

# Load the fine-tuned model and tokenizer
model_dir = "./ner_model"  # Directory where the fine-tuned model is saved
tokenizer = BertTokenizerFast.from_pretrained(model_dir)
model = BertForTokenClassification.from_pretrained(model_dir)

# Set the model to evaluation mode
model.eval()

# Define the string to perform NER on
x = """Title: na Designation: Leader of a transnational trafficking network DOB: 19 Jan. 1983 POB: Sabratha, Libya Good quality a.k.a.: Mus'ab Abu Qarin Low quality a.k.a.: a) ABU-AL QASSIM OMAR Musab Boukrin b) The Doctor c) Al-Grein Nationality: Libya Passport no: a) 782633, issued on 31 May 2005 b) 540794, issued on 12 Jan. 2008 National identification no: na Address: na Listed on: 7 June 2018 Other information: Listed pursuant to paragraphs 15 and 17 of resolution 1970 (Travel Ban, Asset Freeze)"""

# Tokenize the text in variable x
inputs = tokenizer(x, return_tensors="pt", truncation=True, padding=True, is_split_into_words=False)

# Perform NER using the model
with torch.no_grad():
    outputs = model(**inputs)

# Get predictions (logits) and convert them into label IDs
predictions = torch.argmax(outputs.logits, dim=2)

# List of entity labels (ensure it's the same as used during fine-tuning)
label_list = ["O", "B-PERSON", "I-PERSON", "B-PERSON_ARABIC", "I-PERSON_ARABIC", "B-ALIAS", "I-ALIAS",
              "B-DATE_OF_BIRTH", "I-DATE_OF_BIRTH", "B-PLACE_OF_BIRTH", "I-PLACE_OF_BIRTH",
              "B-COUNTRY", "I-COUNTRY", "B-FUNCTION", "I-FUNCTION", "B-RANK", "I-RANK",
              "B-NATIONALITY", "I-NATIONALITY", "B-GENDER", "I-GENDER", "B-PASSPORT_NUMBER",
              "I-PASSPORT_NUMBER", "B-REASON_FOR_LISTING", "I-REASON_FOR_LISTING", "B-DATE_OF_LISTING",
              "I-DATE_OF_LISTING", "B-UN_LISTING_DATE", "I-UN_LISTING_DATE"]

# Convert token IDs back to tokens
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

# Convert predicted label IDs back to label names
predicted_labels = [label_list[p.item()] for p in predictions[0]]

# Clean up the output by handling special tokens and subword tokens
output = []
current_entity = []  # To group multi-token entities like date of birth
previous_label = "O"
previous_token_is_subword = False

for token, label in zip(tokens, predicted_labels):
    # Skip special tokens like [CLS] and [SEP]
    if token in ["[CLS]", "[SEP]"]:
        continue

    # Join subword tokens with the previous token
    if token.startswith("##"):
        if previous_token_is_subword:
            current_entity[-1] = f"{current_entity[-1]}{token[2:]}"
        else:
            current_entity[-1] = f"{current_entity[-1]}{token[2:]}"
        previous_token_is_subword = True
    else:
        # Only include tokens with a relevant entity label
        if label.startswith("B-"):
            if current_entity:
                output.append(" ".join(current_entity))  # Append completed entity
            current_entity = [f"{token} ['{label}']"]  # Start new entity
            previous_label = label
        elif label.startswith("I-") and previous_label[2:] == label[2:]:
            # Only include 'I-' labels if they follow the correct 'B-' label
            current_entity.append(f"{token} ['{label}']")
        else:
            if current_entity:
                output.append(" ".join(current_entity))  # Append completed entity
            output.append(token)  # Normal token without entity
            current_entity = []
            previous_label = "O"
        previous_token_is_subword = False

# Append the last entity if there is one
if current_entity:
    output.append(" ".join(current_entity))

# Join the tokens into a sentence with spaces
output_text = " ".join(output)
print(output_text)


IndexError: list index out of range

import json
import torch
from datasets import load_dataset, load_metric, Dataset
from transformers import BertTokenizerFast, BertForTokenClassification, Trainer, TrainingArguments
from transformers import DataCollatorForTokenClassification

# Load the dataset
def load_ner_dataset(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

dataset_file = 'dummy_ner_dataset_1000.json'
dataset = load_ner_dataset(dataset_file)

# Load the tokenizer and model from Hugging Face
model_checkpoint = "bert-base-cased"
tokenizer = BertTokenizerFast.from_pretrained(model_checkpoint)

# Convert dataset into Hugging Face Dataset format
def tokenize_and_align_labels(examples, label_all_tokens=True):
    tokenized_inputs = tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128, is_split_into_words=False)

    labels = []
    for i, label in enumerate(examples['entities']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = []
        current_label = 0
        for word_id in word_ids:
            if word_id is None:
                label_ids.append(-100)
            elif word_id != current_label:
                label_ids.append(label[current_label]['label'])
                current_label = word_id
            else:
                label_ids.append(label[current_label]['label'] if label_all_tokens else -100)
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Convert dataset to Hugging Face Dataset object
def convert_to_hf_dataset(data):
    texts = [item['text'] for item in data]
    entities = [item['entities'] for item in data]
    return Dataset.from_dict({'text': texts, 'entities': entities})

hf_dataset = convert_to_hf_dataset(dataset)
tokenized_dataset = hf_dataset.map(tokenize_and_align_labels, batched=True)

# Load the pre-trained BERT model for Token Classification
model = BertForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(labels))

# Set up TrainingArguments for fine-tuning
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Data collator to handle padding of sequences
data_collator = DataCollatorForTokenClassification(tokenizer)

# Define metric for evaluation
metric = load_metric("seqeval")

# Compute metrics for evaluation
def compute_metrics(p):
    predictions, labels = p
    predictions = torch.argmax(predictions, dim=2)

    true_labels = [[labels[i] for i in range(len(pred)) if labels[i] != -100] for pred, labels in zip(predictions, labels)]
    true_predictions = [[pred[i] for i in range(len(pred)) if labels[i] != -100] for pred, labels in zip(predictions, labels)]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

# Set up the Trainer for fine-tuning
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Start training
trainer.train()

# Save the final model and tokenizer
trainer.save_model("./ner_model")
tokenizer.save_pretrained("./ner_model")
